In [ ]:
from import_RNA import *

In [ ]:
import itertools

In [ ]:
import warnings

warnings.filterwarnings('ignore')

# XGB Mapping

## Gluta Types

In [ ]:
P12_FF_glut = sc.read_h5ad('h5ads/P22-10dWD_glut_v2.h5ad')
P22_glut = sc.read_h5ad('h5ads/P22Sham_glut_v3.h5ad')

In [ ]:
test_items, mapping_items = pairwise_map(adata_t0=P12_FF_glut, adata_t1=P22_glut,test_lab='Type',
                                                                            train_lab='Type', 
                                                                          recomp_HVGs=True, union_hvgs=False,
                                         t0_dict=make_dict(P12_FF_glut, 'Type'),
                                         t1_dict=make_dict(P22_glut, 'Type'),
                                        min_cells=100, x_lab='P22 Sham', y_lab='P12_FF Sham')

test_labelsP12_FFvsP22, test_predlabelsP12_FFvsP22, test_prediction_P12_FFvsP22 = test_items
mappingconfmatP12_FFvsP22, mappingxticksP12_FFvsP22, mappingplotP12_FFvsP22 = mapping_items

In [ ]:
test_items, mapping_items = pairwise_map(adata_t0=P12_FF_glut, adata_t1=P22_glut,test_lab='Type',
                                                                            train_lab='Type', 
                                                                          recomp_HVGs=True, union_hvgs=False,
                                         t0_dict=make_dict(P12_FF_glut, 'Type'),
                                         t1_dict=make_dict(P22_glut, 'Type'),
                                        min_cells=100, x_lab='P22 Sham', y_lab='P12_FF Sham')

test_labelsP12_FFvsP22, test_predlabelsP12_FFvsP22, test_prediction_P12_FFvsP22 = test_items
mappingconfmatP12_FFvsP22, mappingxticksP12_FFvsP22, mappingplotP12_FFvsP22 = mapping_items

In [ ]:
test_labelsP12_FFvsP22, test_predlabelsP12_FFvsP22, test_prediction_P12_FFvsP22 = test_items
mappingconfmatP12_FFvsP22, mappingxticksP12_FFvsP22_sub, mappingplotP12_FFvsP22 = mapping_items

In [ ]:
a = make_dict(P22_glut, 'Type')
P12_FF_glut.obs['P22 Mapping Prob'] = np.max(test_prediction_P12_FFvsP22, axis=1)
P12_FF_glut.obs['P22 Mapping Label'] = pd.Categorical(test_predlabelsP12_FFvsP22).rename_categories(list(a.keys()))

P12_FF_glut.shape

type_cols = sn.color_palette('pastel').as_hex()+sn.color_palette('Set2').as_hex()+sn.color_palette('Set3').as_hex()+sn.color_palette('hls').as_hex()+sn.color_palette('husl').as_hex()
sc.set_figure_params(dpi_save=300)
sc.pl.umap(P12_FF_glut, color=['Type',], legend_loc='on data', 
           legend_fontsize=10, palette=type_cols, legend_fontweight='semibold', 
           frameon=False,
           title= 'P22 1d WD: 16,654 Glutamatergic Neurons',
           add_outline=True, save='P22-10dWD_glut')

P22_glut.shape

type_cols = sn.color_palette('pastel').as_hex()+sn.color_palette('Set2').as_hex()+sn.color_palette('Set3').as_hex()+sn.color_palette('hls').as_hex()+sn.color_palette('husl').as_hex()
sc.set_figure_params(dpi_save=300)
sc.pl.umap(P22_glut, color=['Type',], legend_loc='on data', 
           legend_fontsize=10, palette=type_cols, legend_fontweight='semibold', 
           frameon=False,
           title= 'P22: 28,334 Glutamatergic Neurons',
           add_outline=True, save='P22_glut')

In [ ]:
sc.pl.umap(P12_FF_glut, color=['Type','P22 Mapping Prob','P22 Mapping Label', 'Baz1a',
                           'Trpc6', 'Igfn1'], legend_loc='on data', legend_fontsize=12)

In [ ]:
P22_glut.obs['Sub'] = P22_glut.obs['Subclass'].cat.rename_categories(['L2/3', 'L4', 'L4/5IT', 'L5NP', 
                                                                      'L5PT', 'L6CT', 'L6IT', 'L6b'])
P12_FF_glut.obs['Sub'] = P12_FF_glut.obs['Subclass'].cat.rename_categories(['L2/3', 'L4', 'L4/5IT', 'L5NP', 
                                                                      'L5PT', 'L6CT', 'L6IT', 'L6b'])

def freq_scatter(x, y, x_lab, y_lab, hue_, unity_lim, low_lim, save=None):

    if (x.shape[0]!=y.shape[0]): 
        print('x and y are different by', np.abs(x.shape[0]-y.shape[0]), 'categories')
    
    if (y.shape[0]>x.shape[0]):
        y = y[x.index]
    else:
        x = x[y.index]

    df = pd.DataFrame(index=list(x.index.values), columns=[x_lab, y_lab],
                      data=np.transpose(np.array([x.values, y.values])))

    df[hue_] = df.index
    sn.scatterplot(data=df, x=x_lab, y = y_lab, hue=hue_, s=50)
    plt.legend(bbox_to_anchor=(1.01, 1.03), loc='upper left', fontsize=14, ncol=2)
    plt.plot(np.linspace(low_lim,unity_lim), np.linspace(low_lim,unity_lim), ls='--', 
             color='black', linewidth=0.75)
    plt.title('Pearson R: '+str(np.round(sp.stats.pearsonr(x, y)[0], 3)))
    plt.grid(False)
    #plt.loglog()
    if (save !=None):
        plt.savefig(save, bbox_inches='tight')

In [ ]:
c = 0
lims = [0.6, 0.41, 0.25, 0.35]
low_lims = [0.2, 0.1,0,0]
xes = []
yes = []
for i in ['L2/3', 'L4', 'L5', 'L6']:
    print(i)
    a=P22_glut[P22_glut.obs['Sub'].str.startswith(i),:]
    b=P12_FF_glut[P12_FF_glut.obs['Sub'].str.startswith(i),:]
    
    xx = a.obs['Type'].value_counts(normalize=True)
    yy = b.obs['P22 Mapping Label'].value_counts(normalize=True)
    
    #print(b.obs['P22 Mapping Label'].value_counts())
    
    if (i=='L4'):
        freq_scatter(x=xx[0:4], y=yy[0:4],
            x_lab='P22 Type Freqs', y_lab='P12_FF Types Mapping Freq', hue_='Types', unity_lim=lims[c], 
                 low_lim=low_lims[c])
        oh=xx
        my=yy
        xes.append(xx[0:4])
        yes.append(yy[0:4])
        
    
    else:
        oh=xx
        my=yy
        freq_scatter(x=xx[xx.index.str.startswith(i)],
            y=yy[yy.index.str.startswith(i)],
            x_lab='P22 Type Freqs', y_lab='P12_FF Types Mapping Freq', hue_='Types', unity_lim=lims[c], 
                 low_lim=low_lims[c])
        xes.append(xx[xx.index.str.startswith(i)])
        yes.append(yy[yy.index.str.startswith(i)])
        
    plt.legend(ncol=1, bbox_to_anchor=(1,1))
    plt.show()
    c = c +1


In [ ]:
freq_scatter(x=P22_glut.obs['Type'].value_counts(normalize=True),
            y= P12_FF_glut.obs['P22 Mapping Label'].value_counts(normalize=True),
            x_lab='P22 Type Frequency', y_lab='10d AWD Type Frequency', hue_='Types', low_lim=0,unity_lim=0.2, 
            hue_ord = P22_glut.obs['Type'].values.categories)
plt.savefig('figures/gluta_10dWD_freqs.pdf',bbox_inches='tight')

## Gluta Types (subclass-by-subclass)

In [ ]:
P12_FF_subs = [P12_FF_glut[P12_FF_glut.obs.Type.str.startswith('L2/3'),:],
            P12_FF_glut[P12_FF_glut.obs.Type.str.startswith('L4'),:],
            P12_FF_glut[P12_FF_glut.obs.Type.str.startswith('L5'),:],
            P12_FF_glut[P12_FF_glut.obs.Type.str.startswith('L6'),:]]

In [ ]:
P22_subs = [P22_glut[P22_glut.obs.Type.str.startswith('L2/3'),:],
            P22_glut[P22_glut.obs.Type.str.startswith('L4'),:],
            P22_glut[P22_glut.obs.Type.str.startswith('L5'),:],
            P22_glut[P22_glut.obs.Type.str.startswith('L6'),:]]

In [ ]:
a

In [ ]:
print(pd.Categorical(test_predlabelsP12_FFvsP22).value_counts())

In [ ]:
    num_cats_mapped = len(pd.Categorical(test_predlabelsP12_FFvsP22).value_counts())


In [ ]:
len(a)

In [ ]:
P12_FF_subs_mapped = []
for sub_index in range(len(P22_subs)):
    test_items, mapping_items = pairwise_map(adata_t0=P12_FF_subs[sub_index], adata_t1=P22_subs[sub_index],test_lab='Type',
                                                                                train_lab='Type', 
                                                                              recomp_HVGs=True, union_hvgs=False,
                                             t0_dict=make_dict(P12_FF_subs[sub_index], 'Type'),
                                             t1_dict=make_dict(P22_subs[sub_index], 'Type'),
                                            min_cells=100, x_lab='P22 Sham', y_lab='P12_FF Sham')

    test_labelsP12_FFvsP22, test_predlabelsP12_FFvsP22, test_prediction_P12_FFvsP22 = test_items
    mappingconfmatP12_FFvsP22, mappingxticksP12_FFvsP22, mappingplotP12_FFvsP22 = mapping_items
    plt.show()
    num_cats_mapped = len(pd.Categorical(test_predlabelsP12_FFvsP22).value_counts())

    P12_FF_LX = P12_FF_subs[sub_index].copy()
    
    
    a = make_dict(P22_subs[sub_index], 'Type')
    
    
    P12_FF_LX.obs['P22 Mapping Prob'] = np.max(test_prediction_P12_FFvsP22, axis=1)
    
    if (num_cats_mapped>len(a)):
        P12_FF_LX.obs['P22 Mapping Label'] = pd.Categorical(test_predlabelsP12_FFvsP22).rename_categories(list(a.keys())+['Unassigned'])
    else:
        P12_FF_LX.obs['P22 Mapping Label'] = pd.Categorical(test_predlabelsP12_FFvsP22).rename_categories(list(a.keys()))

    P12_FF_LX = P12_FF_LX[P12_FF_LX.obs['P22 Mapping Label']!='Unassigned'].copy()
    P12_FF_subs_mapped.append(P12_FF_LX)
    freq_scatter(x=P22_subs[sub_index].obs.Type.value_counts(True),
                y= P12_FF_LX.obs['P22 Mapping Label'].value_counts(True),
                x_lab='P22 Type Freqs', y_lab='P12_FF Types Mapping Freq', hue_='Types', low_lim=0,unity_lim=0.55, )
    plt.show()

In [ ]:
P22_ctrl = P22_subs[0].concatenate(P22_subs[1:])

In [ ]:
P22_10d  = P12_FF_subs_mapped[0].concatenate(P12_FF_subs_mapped[1:])

In [ ]:
plt.figure()
freq_scatter(x=P22_ctrl.obs.Type.value_counts(True),
                y= P22_10d.obs['P22 Mapping Label'].value_counts(True),
                x_lab='P22 Type Freqs', y_lab='P22 10d-WD Types Mapping Freq', hue_='Types', low_lim=0,unity_lim=0.2, )
plt.savefig('figures/10d_typefreq_glut.pdf', bbox_inches='tight')

In [ ]:
def freq_barplot(adata1, adata2):
    
    freq1 = adata1.obs['P22 Mapping Label'].value_counts(normalize=True)
    freq2 =  adata2.obs['Type'].value_counts(normalize=True)

    freq_df = pd.DataFrame(freq1)
    freq_df['Type'] = freq_df.index
    freq_df['Freq'] = freq_df['P22 Mapping Label']

    del freq_df['P22 Mapping Label']

    freq_df['Age'] = len(freq1)*['10d-WD']

    freq_df22 = pd.DataFrame(freq2)
    freq_df22['Freq'] = freq_df22['Type']
    freq_df22['Type'] = freq_df22.index

    freq_df22['Age'] = len(freq2)*['P22']

    df = pd.concat([freq_df, freq_df22])



    fig, axs = plt.subplots(1,1,figsize=(6.25,1.25))
    pivot_df = df.pivot(index='Age', columns='Type', values='Freq')

    # Plot
    pivot_df.loc[['P22','10d-WD']].plot(kind='barh', stacked=True,width=0.8, ax=axs, color=sn.color_palette('Set2').as_hex(),)

    plt.xlabel('Frequency')
    plt.grid(False)
    plt.legend(bbox_to_anchor=(1,1))
    plt.show()

In [ ]:
P12_FF_subs_mapped[0].obs['P22 Mapping Label'].value_counts(normalize=True)

In [ ]:
0.528561 - 0.484319

In [ ]:
P22_subs[0].obs['Type'].value_counts(normalize=True)

In [ ]:
0.180596 - 0.155567

In [ ]:
a = P22_ctrl.obs['Type'].value_counts(normalize=True)

In [ ]:
b = P22_10d.obs['P22 Mapping Label'].value_counts(normalize=True)

In [ ]:
b[a.index]/a

In [ ]:
freq_barplot(P12_FF_subs_mapped[0], P22_subs[0])

In [ ]:
freq_barplot(P12_FF_subs_mapped[1], P22_subs[1])

In [ ]:
freq_barplot(P12_FF_subs_mapped[2], P22_subs[2])

In [ ]:
freq_barplot(P12_FF_subs_mapped[3], P22_subs[3])

### Save mapped objs

In [ ]:
P12_FF_subs_mapped[0].write_h5ad('h5ads_cmprsd/P12_FF_L23_mappedP22.h5ad')

P22_10d.write_h5ad('h5ads_cmprsd/P12_FF_glut_mappedP22.h5ad')

## GABA Type

In [ ]:
P12_FF_gaba = sc.read_h5ad('h5ads/P22-10dWD_gaba_v2.h5ad')
P22_gaba = sc.read_h5ad('h5ads/P22Sham_gaba_v3.h5ad')

**Had to un-do the change to plotConfusionMatrix() from sub-by-sub analysis**

In [ ]:
#code
#import
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scanpy as sc
import pandas as pd
from harmony import harmonize
from sklearn.metrics.cluster import adjusted_rand_score
import random
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score
from sklearn.utils import shuffle
from anndata import AnnData
from sklearn.cluster import KMeans
from typing import Union, Optional, Tuple, Collection, Sequence, Iterable
from scipy.stats import hypergeom
import sklearn.preprocessing
import seaborn as sn
from random import sample
import pickle
#from matplotlib_venn import venn3, venn3_circles
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples


from matplotlib import gridspec
from sklearn.metrics import confusion_matrix, adjusted_rand_score, roc_curve, auc, classification_report, f1_score, cohen_kappa_score
import plotly.graph_objects as go
from itertools import cycle, islice
from sklearn.preprocessing import label_binarize

import scanpy as sc
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.set_figure_params(dpi=100, dpi_save=200)


import scrublet as scr

from typing import Optional

# import warnings
# warnings.filterwarnings("ignore")

import matplotlib as mpl
from matplotlib import gridspec
import xgboost as xgb
from sklearn.metrics import accuracy_score


sc.settings.verbosity = 0           # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.set_figure_params(dpi=75, dpi_save=200)

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

def test_func():
    print('test')
    
def test2():
    print('test2')
def rem_fem(adata):
    for j in list(adata.obs.Type_nn_dists.values.categories):
        if ('Fem' in j):
            adata = adata[adata.obs['Type_nn_dists']!=j,:]
            adata = adata[adata.obs['Type_leiden']!=j,:]
            adata = adata[adata.obs['Type']!=j,:]
    return adata
def pipeline_short(adata, batch_correct, batch_ID):
                   
    if sp.sparse.issparse(adata.X):
        if np.any(adata.X.A<0):
            raise Exception("Matrix contains negative values")
    else: 
        if np.any(adata.X<0):
            raise Exception("Matrix contains negative values")

    
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, batch_key=batch_ID) #HVGs
    adata.raw = adata
    sc.pp.scale(adata, max_value=10) #scale
    sc.tl.pca(adata, svd_solver='arpack') #run PCA
    
    if (batch_correct):
        Z = harmonize(adata.obsm['X_pca'], adata.obs, batch_key = batch_ID)
        adata.obsm['X_harmony'] = Z
        sc.pp.neighbors(adata, n_neighbors=15, use_rep='X_harmony', n_pcs=40)
        sc.tl.leiden(adata)
        sc.tl.umap(adata)

    else:
        sc.pp.neighbors(adata, n_neighbors=15, n_pcs=40)
        sc.tl.leiden(adata)
        sc.tl.umap(adata)
        
def pre_process(adata, batch_ID):
    
    if sp.sparse.issparse(adata.X):
        if np.any(adata.X.A<0):
            raise Exception("Matrix contains negative values")
    else: 
        if np.any(adata.X<0):
            raise Exception("Matrix contains negative values")
    
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)

    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, batch_key=batch_ID) #HVGs
    adata.raw = adata
    sc.pp.scale(adata, max_value=10) #scale
    sc.tl.pca(adata, svd_solver='arpack') #run PCA
    

def pipeline(adata, 
             batch_correct: Optional[bool] = None,
             batch_ID: Optional[str] = None):
    
    if (batch_correct):
        Z = harmonize(adata.obsm['X_pca'], adata.obs, batch_key = batch_ID)
        adata.obsm['X_harmony'] = Z
        sc.pp.neighbors(adata, n_neighbors=15, use_rep='X_harmony', n_pcs=40)
        sc.tl.leiden(adata)
        sc.tl.umap(adata)

    else:
        sc.pp.neighbors(adata, n_neighbors=15, n_pcs=40)
        sc.tl.leiden(adata)
        sc.tl.umap(adata)

def clust_obs_plot(adata, obs_id, obs_id_split):

    obs_quants = []
    for i in adata.obs[obs_id_split].values.categories:
    #for i in dr_corr.obs['Atlas Label'].values.categories:
        obs_quants.append(np.mean(adata[adata.obs[obs_id_split]==i].obs[obs_id].values))
        
    x, y = adata.obs[obs_id_split].values.categories, obs_quants
    ser = pd.Series(y,x).sort_values(ascending=False)
    
    x,y = ser.index, ser.values
    
    plt.figure(figsize=(15,4))
    plt.bar(x,y)
    plt.axhline(np.median(adata.obs[obs_id]), color='r')
    plt.ylabel(obs_id)
    
    return x, y

def DE(adata, obs_id, obs_id_test, ref, pts_thresh, lf_thresh):

    sc.tl.rank_genes_groups(adata, groupby=obs_id, groups=[obs_id_test], 
                                reference=ref, method='t-test', pts=True, use_raw=True)

    lfcs = adata.uns['rank_genes_groups']['logfoldchanges'].astype([(obs_id_test, '<f8')]).view('<f8') 

    l231_genes = adata.uns['rank_genes_groups']['pts']

    lfcs = []
    p_adj = []
    names = list(adata.uns['rank_genes_groups']['names'].astype([(obs_id_test, '<U50')]).view('<U50'))
    logfoldchanges = adata.uns['rank_genes_groups']['logfoldchanges'].astype([(obs_id_test, '<f8')]).view('<f8')
    pvals_adj = adata.uns['rank_genes_groups']['pvals_adj'].astype([(obs_id_test, '<f8')]).view('<f8')

    for i in l231_genes.index:
        lfcs.append(logfoldchanges[names.index(i)])
        p_adj.append(pvals_adj[names.index(i)])

    l231_genes['LF'] = lfcs
    l231_genes['p_adj'] = p_adj
    
    #plt.hist(l231_genes[obs_id_test].values)

    l231_genes = l231_genes[l231_genes[obs_id_test]>pts_thresh]
    
    sort_LF = l231_genes.sort_values('LF', ascending=False)
    
    a = np.where(sort_LF[sort_LF['LF']>0.6]['p_adj'].values<0.05)[0].shape[0]
    b = sort_LF[sort_LF['LF']>0.6].shape[0]
    if(a == b):
        print('cutoffs are good at 1.5 FC level')
    else:
        print(a,b)
        
    return sort_LF[sort_LF['LF']>lf_thresh]
        
def clust_obs(adata, obs_id, obs_id_split):

    obs_quants = []
    for i in adata.obs[obs_id_split].values.categories:
    #for i in dr_corr.obs['Atlas Label'].values.categories:
        obs_quants.append(np.mean(adata[adata.obs[obs_id_split]==i].obs[obs_id].values))
        
    x, y = adata.obs[obs_id_split].values.categories, obs_quants
    ser = pd.Series(y,x).sort_values(ascending=False)
    return ser.index, ser.values

def res_tune(adata, def_labels):
    
    adata_ari = []
    adata_num_clusts = []
    for res in np.arange(1,2.1, 0.1):
        sc.tl.leiden(adata, resolution=res)

        adata_num_clusts.append(len(adata.obs.leiden.values.categories))
        adata_ari.append(adjusted_rand_score(def_labels, list(adata.obs.leiden.values)))
        
    return adata_num_clusts, adata_ari

from math import log, e
def entropy2(labels, base=None):
  """ Computes entropy of label distribution. """

  n_labels = len(labels)

  if n_labels <= 1:
    return 0

  value,counts = np.unique(labels, return_counts=True)
  probs = counts / n_labels
  n_classes = np.count_nonzero(probs)

  if n_classes <= 1:
    return 0

  ent = 0.

  # Compute entropy
  base = e if base is None else base
  for i in probs:
    ent -= i * log(i, base)

  return ent

def check_clust(adata, clus, ref_age):
    sc.pl.umap(adata[adata.obs.leiden==clus], color=[ref_age+'-2022 Mapping Prob', ref_age+'-2022 Mapping Label'])
    print(adata[adata.obs.leiden==clus].shape, 'mean prob', np.mean(adata[adata.obs.leiden==clus].obs[ref_age+'-2022 Mapping Prob']))
    ser = adata[adata.obs.leiden==clus].obs['Sample'].value_counts()
    plt.bar(ser.index, ser.values,)
    plt.xticks(rotation=90)
    plt.show()
    ser = adata[adata.obs.leiden==clus].obs[ref_age+'-2022 Mapping Label'].value_counts()
    plt.bar(ser.index, ser.values,)
    plt.xticks(rotation=90)
    plt.show()
    
def maj_vote_annot(adata, clust_id, putative_annot, final_annot, vlmcendo=False):
    clust_list = []
    for i in adata.obs[clust_id].values.categories:
        clust = adata[adata.obs[clust_id]==i,:]
        clust_df = clust.obs[putative_annot].value_counts()
        if (clust_df.index[0]=='Unassigned'):
            biggest_cat = clust_df.index[1]
        else: biggest_cat = clust_df.index[0]
        if (vlmcendo):
            if (biggest_cat in ('VLMC', 'Endo')):
                clust.obs[putative_annot+'_maj'] = ['VLMC+Endo']*clust.shape[0]
                clust_list.append(clust)
            else:
                clust.obs[putative_annot+'_maj'] = [biggest_cat]*clust.shape[0]
                clust_list.append(clust)
        else:
            clust.obs[putative_annot+'_maj'] = [biggest_cat]*clust.shape[0]
            clust_list.append(clust)

    adata_annot = clust_list[0].concatenate(clust_list[1:], index_unique=None)
    adata_annot.obs[final_annot] = adata_annot.obs[putative_annot+'_maj']
    a = sc.pl.dotplot(adata_annot, 'Mdga1', 'leiden',) #to fix categories
    return adata_annot

def clus_sample_bars(adata, a_, b_, samp_id, clus_id, size=(13,9.5)):
    adata_ser = adata.obs[samp_id].value_counts(normalize=True)
    plt.bar(adata_ser.index, adata_ser.values)
    plt.xticks(rotation=90)
    plt.title('Overall Dataset')
    plt.show()
    a,b = 0 ,0 
    fig, axs = plt.subplots(a_,b_, figsize=size)

    for i in adata.obs[clus_id].values.categories:
        clus = adata[adata.obs[clus_id]==i,:]
        clus_ser = clus.obs[samp_id].value_counts(normalize=True)
        axs[a,b].bar(clus_ser.index, clus_ser.values)
        axs[a,b].set_title(i+' ('+str(clus.shape[0])+' cells)')
        axs[a,b].set_ylabel('Fraction')
        axs[a,b].set_xticklabels(clus_ser.index, rotation=90)
        b = b + 1
        if (b>=b_):
            b = 0
            a = a + 1
    plt.tight_layout()
    
    
def euclidean_distance(vector1, vectors_list):
    # Ensure that the input vectors are numpy arrays
    vector1 = np.array(vector1)
    vectors_list = [np.array(vector) for vector in vectors_list]
    
    # Calculate the Euclidean distance between the input vector and each vector in the list
    distances = [np.linalg.norm(vector1 - vector) for vector in vectors_list]
    return distances

def nn_voting(adata, type_old, type_new, delta_thresh):
    adata.obs['idx'] = np.arange(adata.shape[0])

#     #re-make graph using UMAP space for voting below
#     sc.pp.neighbors(adata, n_neighbors=15, use_rep='X_umap')
#     sc.tl.umap(adata)

    adata.obs[type_new] = adata.obs[type_old]
    adata_un_idx = adata[adata.obs[type_old]=='Unassigned',:].obs.idx.values #indices of unassigned cells
    neigh_matx = adata.obsp['distances'].A #each row's nonzero entries tells neighbors
    adata_iGBs = list(adata.obs[type_old].values) #all the step1 assignments
    n_neighbs = adata.uns['neighbors']['params']['n_neighbors']
    print(adata.uns['neighbors']['params'])
    print(' ')
    
    print("Pre-voting unassigned: ", adata_iGBs.count('Unassigned'),
          adata_iGBs.count('Unassigned')/adata.shape[0])
    
    
    delta = 1 #init delta
    while (delta>delta_thresh):    
        un_frac1 = adata_iGBs.count('Unassigned')/adata.shape[0] #frac of unassigned cells before voting

        #loop thru each cell
        for i in adata_un_idx:

            #so that it only loops thru still-unassigned cells after first pass
            if (adata.obs[type_new][i]=='Unassigned'):
                neighbs_idx = np.where(neigh_matx[i,:]>0)[0] #cell i's neighbors
                neighbs_iGBs = adata[neighbs_idx].obs[type_new] #the neighbors' iGBs

#                 #if there's a type in the neighbors that is majority, assign it
#                 if (neighbs_iGBs.value_counts()[0] > n_neighbs/2):
#                     adata_iGBs[i] = neighbs_iGBs.value_counts().index[0]
                
                #no need to be majority, just be biggest one
                adata_iGBs[i] = neighbs_iGBs.value_counts().index[0]
                #update the IDs to help assignment of next cell
                adata.obs[type_new] =  pd.Categorical(adata_iGBs)

        un_frac2 = adata_iGBs.count('Unassigned')/adata.shape[0] #frac of unassigned cells after voting

        delta = (un_frac1-un_frac2)/un_frac1   #stop when this changes by less than 1%
        print(delta, un_frac2)

    print("Post-voting unassigned: ", adata_iGBs.count('Unassigned'),
          adata_iGBs.count('Unassigned')/adata.shape[0])
    
    return adata


def make_hmap(adata,thres, y_obs='Type', x_obs='leiden', ):
    crosstab_data = pd.crosstab(adata.obs[y_obs], adata.obs[x_obs], 
                           normalize='index')
    # For rows
    row_order = np.argsort(-crosstab_data.values.max(axis=1))

    # For columns
    column_order = np.argsort(-crosstab_data.values.max(axis=0))

    reordered_rows = crosstab_data.index[row_order]
    reordered_crosstab_rows = crosstab_data.iloc[row_order]

    # For columns
    reordered_columns = crosstab_data.columns[column_order]
    reordered_crosstab = reordered_crosstab_rows[reordered_columns]
    #return reordered_crosstab
    
    
    from sklearn.metrics import adjusted_rand_score

    

    labels_true = adata.obs[y_obs]
    labels_pred = adata.obs[x_obs]

    # Compute the Adjusted Rand Index
    ari = adjusted_rand_score(labels_true, labels_pred)
    
    return [reordered_crosstab.applymap(lambda value: value if value > thres else np.round(value, 2)),
           ari]
def freq_scatter(x, y, x_lab, y_lab, hue_, unity_lim, low_lim):

    if (x.shape[0]!=y.shape[0]): 
        print('x and y are different by', np.abs(x.shape[0]-y.shape[0]), 'categories')
    
    if (y.shape[0]>x.shape[0]):
        y = y[x.index]
    else:
        x = x[y.index]

    df = pd.DataFrame(index=list(x.index.values), columns=[x_lab, y_lab],
                      data=np.transpose(np.array([x.values, y.values])))

    df[hue_] = df.index
    sn.scatterplot(data=df, x=x_lab, y = y_lab, hue=hue_, s=50)
    plt.legend(bbox_to_anchor=(1.01, 1.03), loc='upper left', fontsize=14, ncol=2)
    plt.plot(np.linspace(low_lim,unity_lim), np.linspace(low_lim,unity_lim), ls='--', 
             color='black', linewidth=0.75)
    plt.title('Pearson R: '+str(np.round(sp.stats.pearsonr(x, y)[0], 3)))
    plt.grid(False)
    #plt.loglog()
    


from anndata import AnnData
from typing import Union, Optional, Tuple, Collection, Sequence, Iterable

def module_score(adata:AnnData, genes_use: list, score_name: Optional[str] = None, verbose: bool = True):
    
    """\
    Compute module scores for all cells in adata as described in methods of RGC-dev paper.
    
    
    Parameters
    ----------
    adata
        The (annotated) data matrix of shape `n_obs` × `n_vars`.
        Rows correspond to cells and columns to genes.
    genes_use
        list of genes in module of interest
    score_name
        Name endowed to the module score to be computed
        e.g. "Mod1"
    verbose
        Inform user of fraction of module genes that are in adata
        
    Returns
    -------
    adata with a new .obs called score_name
    
    """
    
    if (score_name==None):
        score_name = str(input("Provide a name for this score (no spaces): "))
        
    genes_use0 = genes_use
    genes_use = list(set(genes_use).intersection(adata.var_names))#genes that are both in module and `adata`
    
     
    if (len(genes_use) == 0):
        raise ValueError("Error : Must provide a list of genes that are present in the data")
        
    
    if (verbose):
        if(len(genes_use0) > len(genes_use)):
            n = len(genes_use0) - len(genes_use)
            print(score_name,": Note that", n, "of the", len(genes_use0), "genes in your module do not exist in the data set." )
        else:
            print(score_name,": Note that all of the", len(genes_use), "genes in your module are in the data set." )
    
    
    
    adata_score = adata.copy()
    adata_score = adata[:,genes_use]
    
    counts_modgenes = adata_score.X.toarray() #all cells, module genes
    counts_all = adata.X.toarray() #all cells, all genes
    #scores = np.mean(counts_modgenes, axis=1) - np.mean(counts_all, axis=1) #(row means of counts_modgenes ) - (row means of counts_all)
    scores = np.mean(counts_modgenes, axis=1) #(row means of counts_modgenes )

    adata.obs[score_name] = scores
    
    return genes_use    
    

def clean_subclasses(adata, rep):
    #define avg PC position of each type. Gotta use a type with no "Unassigned" group
    typ_mean_dict = {}
    for i in adata.obs['Subclass'].values.categories:
        typ_mean_dict[i] = np.mean(adata[adata.obs['Subclass']==i,:].obsm[rep][:,0:40], axis=0)

    #assign based on proximity to avg type
    dists_list = []
    dict_keys = list(typ_mean_dict.keys())
    for i in range(adata.shape[0]):
        typ_ = adata.obs['Subclass'][i]
        dists = euclidean_distance(adata.obsm[rep][i,0:40], list(typ_mean_dict.values()))

        #dist = dists[dict_keys.index(typ_)]
        typ = dict_keys[np.argmin(dists)]
        #types.append(typ)
        if (typ!=typ_):
            dists_list.append(100)
        else: dists_list.append(0)

    adata.obs['Dist to Subclass'] = dists_list
    
def make_dict(adata, obs_id):

    adata_dict = {}
    for num,i in enumerate(adata.obs[obs_id].values.categories):
        adata_dict[i] = num
        
    #adata_dict['Unassigned'] = num + 1
    
    return adata_dict
    

def plotConfusionMatrix(
    ytrue,
    ypred,
    type,
    xaxislabel,
    yaxislabel,
    title,
    train_dict,
    test_dict=None,
    re_order=None,
    re_order_cols = None,
    re_index = None,
    re_order_rows = None,
    save_as=None,):
    
    #very bad
    numbertrainclasses = len(set(ypred))
    numbertestclasses = len(set(ytrue))
    
    #cfm is 11x11 b/c 11 is = y_true U y_pred
    confusionmatrix = confusion_matrix(y_true = ytrue, y_pred = ypred)
        
    #only need this when mapping b/c if validaiton, all classes will be used and cfm will be constructed properly
    if type == 'mapping':
        rows = np.where(np.sum(confusionmatrix, axis=1)>0)[0]
        cols = np.where(np.sum(confusionmatrix, axis=0)>0)[0]
        
        cfm = confusionmatrix[rows,:][:,cols]
        
        #show all columns, even ones with no mapping
        
        #this was changed 10/12/2023 to address unassigned issue
        cfm_z = np.zeros((len(test_dict),len(train_dict)))
        #cfm_z = np.zeros((len(rows),len(cols)))

        cfm_z[:, np.array(pd.Categorical(ypred).categories, dtype='int')]=cfm 
        confusionmatrix = cfm_z
        
        #always keep only as many as rows as num of test classes
        #but, b/c of python's 0 indexing, if the number of training classes is in the 
        #y_pred list, then that means there was an unassigned
#       if numbertrainclasses in ypred:
#         confusionmatrix = confusionmatrix[0:numbertestclasses,0:numbertrainclasses+1]#for Unassigned
#       else:
#         confusionmatrix = confusionmatrix[0:numbertestclasses,0:numbertrainclasses]
    
        confmatpercent = confusionmatrix/np.sum(confusionmatrix, axis=1).reshape(-1,1)

        conf_df = pd.DataFrame(confmatpercent)
        conf_df.index = list(test_dict.keys())

        #name columns of conf mat
        if(len(conf_df.columns)>len(train_dict)):
            conf_df.columns = list(train_dict.keys())+['Unassigned']
        else:
            conf_df.columns = list(train_dict.keys())


        if (re_order):
            conf_df = conf_df[re_order_cols]
        if (re_index):
            conf_df = conf_df.reindex(re_order_rows)

        diagcm = conf_df.to_numpy()
    
    
        xticksactual = list(conf_df.columns)
        
        #print(conf_df)
        
    
    else:
        confmatpercent = np.zeros(confusionmatrix.shape)
        for i in range(confusionmatrix.shape[0]):
            if np.sum(confusionmatrix[i,:]) != 0:
                confmatpercent[i,:] = confusionmatrix[i,:]/np.sum(confusionmatrix[i,:])
            else:
                confmatpercent[i,:] = confusionmatrix[i,:]
            diagcm = confmatpercent
            xticks = np.linspace(0, confmatpercent.shape[1]-1, confmatpercent.shape[1], dtype = int)
        xticksactual = []
        for i in xticks:
            if i != numbertrainclasses:
                xticksactual.append(list(train_dict.keys())[i])
            else:
                xticksactual.append('Unassigned')
        
    dot_max = np.max(diagcm.flatten())
    dot_min = 0
    if dot_min != 0 or dot_max != 1:
        frac = np.clip(diagcm, dot_min, dot_max)
        old_range = dot_max - dot_min
        frac = (frac - dot_min) / old_range
    else:
        frac = diagcm
    xvalues = []
    yvalues = []
    sizes = []
    for i in range(diagcm.shape[0]):
        for j in range(diagcm.shape[1]):
            xvalues.append(j)
            yvalues.append(i)
            sizes.append((frac[i,j]*35)**1.5)
    size_legend_width = 0.5
    height = diagcm.shape[0] * 0.3 + 1
    height = max([1.5, height])
    heatmap_width = diagcm.shape[1] * 0.35
    width = (
        heatmap_width
        + size_legend_width
        )
    fig = plt.figure(figsize=(width, height))
    axs = gridspec.GridSpec(
        nrows=2,
        ncols=2,
        wspace=0.02,
        hspace=0.04,
        width_ratios=[
                    heatmap_width,
                    size_legend_width
                    ],
        height_ratios = [0.5, 10]
        )
    dot_ax = fig.add_subplot(axs[1, 0])
    dot_ax.scatter(xvalues,yvalues, s = sizes, c = 'blue', norm=None, edgecolor='none')
    y_ticks = range(diagcm.shape[0])
    dot_ax.set_yticks(y_ticks)
    if type == 'validation':
        dot_ax.set_yticklabels(list(train_dict.keys()))
    elif type == 'mapping':
      #dot_ax.set_yticklabels(list(test_dict.keys()))
        dot_ax.set_yticklabels(list(conf_df.index))
    x_ticks = range(diagcm.shape[1])
    dot_ax.set_xticks(x_ticks)
    dot_ax.set_xticklabels(xticksactual, rotation=90)
    dot_ax.tick_params(axis='both', labelsize='small')
    dot_ax.grid(True, linewidth = 0.2)
    dot_ax.set_axisbelow(True)
    dot_ax.set_xlim(-0.5, diagcm.shape[1] + 0.5)
    ymin, ymax = dot_ax.get_ylim()
    dot_ax.set_ylim(ymax + 0.5, ymin - 0.5)
    dot_ax.set_xlim(-1, diagcm.shape[1])
    dot_ax.set_xlabel(xaxislabel)
    dot_ax.set_ylabel(yaxislabel)
    dot_ax.set_title(title)
    size_legend_height = min(1.75, height)
    wspace = 10.5 / width
    axs3 = gridspec.GridSpecFromSubplotSpec(
        2,
        1,
        subplot_spec=axs[1, 1],
        wspace=wspace,
        height_ratios=[
                    size_legend_height / height,
                    (height - size_legend_height) / height
                    ]
        )
    diff = dot_max - dot_min
    if 0.3 < diff <= 0.6:
        step = 0.1
    elif diff <= 0.3:
        step = 0.05
    else:
        step = 0.2
    fracs_legends = np.arange(dot_max, dot_min, step * -1)[::-1]
    if dot_min != 0 or dot_max != 1:
        fracs_values = (fracs_legends - dot_min) / old_range
    else:
        fracs_values = fracs_legends
    size = (fracs_values * 35) ** 1.5
    size_legend = fig.add_subplot(axs3[0])
    size_legend.scatter(np.repeat(0, len(size)), range(len(size)), s=size, c = 'blue')
    size_legend.set_yticks(range(len(size)))
    labels = ["{:.0%}".format(x) for x in fracs_legends]
    if dot_max < 1:
        labels[-1] = ">" + labels[-1]
    size_legend.set_yticklabels(labels)
    size_legend.set_yticklabels(["{:.0%}".format(x) for x in fracs_legends])
    size_legend.tick_params(axis='y', left=False, labelleft=False, labelright=True)
    size_legend.tick_params(axis='x', bottom=False, labelbottom=False)
    size_legend.spines['right'].set_visible(False)
    size_legend.spines['top'].set_visible(False)
    size_legend.spines['left'].set_visible(False)
    size_legend.spines['bottom'].set_visible(False)
    size_legend.grid(False)
    ymin, ymax = size_legend.get_ylim()
    size_legend.set_ylim(ymin, ymax + 0.5)
    
    if (save_as is not None):
        fig.savefig(save_as, bbox_inches = 'tight')

    return diagcm, xticksactual, axs



#This helper method plots validation plots in sequential order (i.e. first plot is for first batch, second plot is for second batch, etc.)
def plot_validation_plots(validation_label_train_70, valid_predlabels_train_70, train_dict, save_as=None):
    
    ARI = adjusted_rand_score(labels_true = validation_label_train_70, 
                              labels_pred = valid_predlabels_train_70)
    
    
    c = 0
    for i in range(validation_label_train_70.shape[0]):
        if (validation_label_train_70[i]!=valid_predlabels_train_70[i]):
            c = c +1
    acc = (1 - c/len(validation_label_train_70))*100
    
           
    validationconfmat, validationxticks, validationplot = plotConfusionMatrix(
    ytrue = validation_label_train_70,
    ypred = valid_predlabels_train_70,
    train_dict=train_dict,
    type = 'validation',
    save_as = save_as,
    title = 'ARI = {:.3f}, Accuracy = {:.3f}'.format(ARI, acc),
    xaxislabel = 'Predicted',
    yaxislabel = 'True'
    )

def plot_mapping(test_labels, test_predlabels, test_dict, train_dict, 
                 xaxislabel, yaxislabel,
                re_order=None,
    re_order_cols = None,
                 re_index = None,
    re_order_rows = None, save_as=None):
    
    ARI = adjusted_rand_score(labels_true = test_labels, 
                              labels_pred = test_predlabels)
    NCE = calculateNCE(labels_true = test_labels, labels_pred = test_predlabels)
    
 
    
           
    mappingconfmat, mappingxticks, mappingplot = plotConfusionMatrix(
    ytrue = test_labels,
    ypred = test_predlabels,
    test_dict=test_dict,
    train_dict=train_dict,
    type = 'mapping',
    save_as = save_as,
    title = 'ARI = {:.3f}, NCE = {:.3f}'.format(ARI, NCE),
    xaxislabel =xaxislabel,
    yaxislabel = yaxislabel,
        re_order=re_order,
    re_order_cols = re_order_cols,
        re_index = re_index,
    re_order_rows = re_order_rows,
    ) 
    return mappingconfmat, mappingxticks, mappingplot
      
      
#This helper method uses xgboost to train classifiers.
def trainclassifier(train_anndata, common_top_genes, obs_id, train_dict, eta,
                    max_cells_per_ident, train_frac, min_cells_per_ident):
    
    if sp.sparse.issparse(train_anndata.X):
        if np.any(train_anndata.X.A<0):
            raise Exception("Matrix contains negative values")
    else: 
        if np.any(train_anndata.X<0):
            raise Exception("Matrix contains negative values")

    
    start_time = time.time()
    
    numbertrainclasses = len(train_anndata.obs[obs_id].values.categories)

    xgb_params_train = {
            'objective':'multi:softprob',
            'eval_metric':'mlogloss',
            'num_class':numbertrainclasses,
            'eta':eta,
            'max_depth':4,
            'subsample': 0.6}
    nround = 200
    #Train XGBoost on 70% of training data and validate on the remaining data


    training_set_train_70 = []
    validation_set_train_70 = []
    training_label_train_70 = []
    validation_label_train_70 = []

    #loop thru classes to split for training and validation
    for i in train_anndata.obs[obs_id].values.categories:
        
        #how many cells in a class
        cells_in_clust = train_anndata[train_anndata.obs[obs_id]==i,:].obs_names #cell names
        n = min(max_cells_per_ident,round(len(cells_in_clust)*train_frac))
        
        #sample 70% for training and rest for validation
        train_temp = np.random.choice(cells_in_clust,n,replace = False)
        validation_temp = np.setdiff1d(cells_in_clust, train_temp)
        
        #upsample small clusters
        if len(train_temp) < min_cells_per_ident:
            train_temp_bootstrap = np.random.choice(train_temp, size = min_cells_per_ident - int(len(train_temp)))
            train_temp = np.hstack([train_temp_bootstrap, train_temp])
        
        #store training and validation **names** of cells in vectors, which update for every class
        training_set_train_70 = np.hstack([training_set_train_70,train_temp])
        validation_set_train_70 = np.hstack([validation_set_train_70,validation_temp])
        
        #store training and validation **labels** of cells in vectors, which update for every class
        training_label_train_70 = np.hstack([training_label_train_70,np.repeat(train_dict[i],len(train_temp))])
        validation_label_train_70 = np.hstack([validation_label_train_70,np.repeat(train_dict[i],len(validation_temp))])

        #need train_dict b/c XGboost needs number as class labels, not words
        #this is only deconvulted later in plotting function
        
    #put data in XGB format
    X_train = train_anndata[training_set_train_70,common_top_genes].X
    train_matrix_train_70 = xgb.DMatrix(data = X_train, label = training_label_train_70, 
                                        feature_names = common_top_genes)
    
    X_valid = train_anndata[validation_set_train_70,common_top_genes].X
    validation_matrix_train_70 = xgb.DMatrix(data = X_valid, label = validation_label_train_70, 
                                             feature_names = common_top_genes)

    del training_set_train_70, validation_set_train_70, training_label_train_70
    
    #Train on 70%
    bst_model_train_70 = xgb.train(
        params = xgb_params_train,
        dtrain = train_matrix_train_70,
        num_boost_round = nround)
    
    #Validate on 30%
    #a validation_cells x numclasses matrix, with each vector containing prob association with the classes
    validation_pred_train_70 = bst_model_train_70.predict(data = validation_matrix_train_70)
    
    #for each cell, go through vec of probs and take index of max prob: that's assignment
    valid_predlabels_train_70 = np.zeros((validation_pred_train_70.shape[0]))
    for i in range(validation_pred_train_70.shape[0]):
        valid_predlabels_train_70[i] = np.argmax(validation_pred_train_70[i,:])
        
    
    #Train on 100%
    #Train XGBoost on the full training data
    training_set_train_full = []
    training_label_train_full = []

    for i in train_anndata.obs[obs_id].values.categories.values:
        train_temp = train_anndata.obs.index[train_anndata.obs[obs_id].values == i]
        if len(train_temp) < 100:
            train_temp_bootstrap = np.random.choice(train_temp, size = 100 - int(len(train_temp)))
            train_temp = np.hstack([train_temp_bootstrap, train_temp])
        
        #indices of cells in class
        training_set_train_full = np.hstack([training_set_train_full,train_temp])
        
        #labels of cells in class: [label*N_class] stacked onto previous classes
        training_label_train_full = np.hstack([training_label_train_full,np.repeat(train_dict[i],len(train_temp))])


    X_train_full = train_anndata[training_set_train_full,common_top_genes].X
    full_training_data = xgb.DMatrix(data = X_train_full, label = training_label_train_full, 
                                     feature_names = common_top_genes)

    del training_set_train_full, training_label_train_full

    bst_model_full_train = xgb.train(
        params = xgb_params_train,
        dtrain = full_training_data,
        num_boost_round = nround)

    
    
    print('trainclassifier() complete after', np.round(time.time() - start_time), 'seconds')
    
    
    f1 = f1_score(validation_label_train_70, valid_predlabels_train_70, average = None)

    
    #real labels of validation set, predicted labels, classifier.
    #recall these are all integers that are deconvulted later in plotting using the dicts
    return validation_label_train_70, valid_predlabels_train_70, bst_model_full_train, f1


#This helper method predicts the testing cluster labels.
def predict(train_anndata, common_top_genes, bst_model_train_full, test_anndata, 
            train_obs_id, test_dict, test_obs_id):
    
    
    if sp.sparse.issparse(train_anndata.X):
        if np.any(train_anndata.X.A<0):
            raise Exception("Training matrix contains negative values")
    else: 
        if np.any(train_anndata.X<0):
            raise Exception("Training matrix contains negative values")

    if sp.sparse.issparse(test_anndata.X):
        if np.any(test_anndata.X.A<0):
            raise Exception("Testing matrix contains negative values")
    else: 
        if np.any(test_anndata.X<0):
            raise Exception("Testing matrix contains negative values")
  
    
    #Predict the testing cluster labels
    #how many classes mapping to 
    numbertrainclasses = len(train_anndata.obs[train_obs_id].values.categories)
    
    #put testing data into XGB format
    full_testing_data = xgb.DMatrix(data = test_anndata[:,common_top_genes].X, 
                                    feature_names=common_top_genes)
    
    #a testing_cells x numclasses matrix, with each vector containing prob association with the classes
    test_prediction = bst_model_train_full.predict(data = full_testing_data)

    #for each cell, go through vec of probs and take index of max prob (if greater than ...): that's assignment

    
    test_predlabels = np.zeros((test_prediction.shape[0]))
    for i in range(test_prediction.shape[0]):
        if np.max(test_prediction[i, :]) > 1.1*(1/numbertrainclasses):
            test_predlabels[i] = np.argmax(test_prediction[i,:])
        
        #"unassigned" is a label one larger than all b/c python begins indexing at 0
        else:
            test_predlabels[i] = numbertrainclasses
        
    test_labels = np.zeros(len(test_anndata.obs[test_obs_id].values))
    for i,l in enumerate(test_anndata.obs[test_obs_id].values):
        test_labels[i] = test_dict[l]

    #actual labels of testing set, the labels that test set mapped to 
    return test_labels, test_predlabels, test_prediction

def calculateNCE(labels_true,labels_pred):
    X = labels_true
    Y = labels_pred
    contTable = confusion_matrix(X,Y)[0:len(np.unique(X)), 0:len(np.unique(Y))]
    a = np.sum(contTable, axis = 1)
    b = np.sum(contTable, axis = 0)
    N = np.sum(contTable)
    pij = contTable/N
    pi = a/N
    pj = b/N
    Hyx = np.zeros(contTable.shape)
    for i in range(contTable.shape[0]):
        for j in range(contTable.shape[1]):
          if pij[i,j] == 0:
            Hyx[i,j] = 0
          else:
            Hyx[i,j] = pij[i,j]*np.log10(pij[i,j]/pi[i])
    CE = -np.sum(Hyx)
    Hyi = np.zeros(contTable.shape[1])
    for j in range(contTable.shape[1]):
      if pj[j] == 0:
       Hyi[j] = 0
      else:
        Hyi[j] = pj[j]*np.log10(pj[j])
    Hy = -np.sum(Hyi)
    NCE = CE/Hy
    return NCE

def train_validate(adata, adata_cell,preproc=False):
    if (preproc):
        adata_cell.X = adata_cell.raw.X
        sc.pp.highly_variable_genes(adata_cell, min_mean=0.0125, max_mean=3, min_disp=0.5) #HVGs

    common_hvgs = list(set(adata[:,adata.var.highly_variable].var_names).intersection(set(adata_cell.var_names)))
    adata_m_dict = make_dict(adata, obs_id='leiden')
    adata_cell_dict = make_dict(adata_cell, obs_id='Subclass')
    print(len(common_hvgs), 'Shared HVGs')

    valid_truelabel_adata_cell, valid_predlabels_adata_cell, model_atlas_adata_cell = trainclassifier(train_anndata=adata_cell, 
                                                                                                 common_top_genes=common_hvgs, 
                                                                                                 obs_id='Subclass', 
                                                                                                 train_dict=adata_cell_dict, 
                                                                                                 eta=0.2,
                                                                                                 max_cells_per_ident=1000, 
                                                                                          train_frac=0.7, 
                                                                                                 min_cells_per_ident=100)

    plot_validation_plots(valid_truelabel_adata_cell, valid_predlabels_adata_cell, train_dict=adata_cell_dict)
    
    return model_atlas_adata_cell

def pairwise_map(adata_t0, adata_t1, test_lab, train_lab, t0_dict, t1_dict, recomp_HVGs, min_cells,
                x_lab, y_lab, union_hvgs):
    adata_t0 = adata_t0.copy() 
    adata_t1 = adata_t1.copy() 
    adata_t0.X = adata_t0.raw.X
    adata_t1.X = adata_t1.raw.X
    
    if (recomp_HVGs):
        sc.pp.highly_variable_genes(adata_t0, min_mean=0.0125, max_mean=3, min_disp=0.5) #HVGs
        sc.pp.highly_variable_genes(adata_t1, min_mean=0.0125, max_mean=3, min_disp=0.5) #HVGs
        t0_hvgs = list(adata_t0[:, adata_t0.var.highly_variable].var_names)
        t1_hvgs = list(adata_t1[:, adata_t1.var.highly_variable].var_names)
        
        if (union_hvgs):
            t0_t1hvgs_ = list(set(t0_hvgs).union(t1_hvgs))
            data_inter = list(set(adata_t1.var_names).intersection(set(adata_t0.var_names)))
            t0_t1hvgs = list(set(data_inter).intersection(set(t0_t1hvgs_)))

        else:
            t0_t1hvgs = list(set(t0_hvgs).intersection(t1_hvgs))

    
    else:
        t0_hvgs = list(adata_t0[:, adata_t0.var.highly_variable].var_names)
        t1_hvgs = list(adata_t1[:, adata_t1.var.highly_variable].var_names)
        t0_t1hvgs = list(set(t0_hvgs).intersection(t1_hvgs))

    print(len(t0_t1hvgs), 'shared HVGs')
    
    validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, model_t1,model_f1 = trainclassifier(train_anndata=adata_t1, 
                                                                                                 common_top_genes=t0_t1hvgs, 
                                                                                                 obs_id=train_lab, 
                                                                                                 train_dict=t1_dict, 
                                                                                                 eta=0.2,
                                                                                                 max_cells_per_ident=1000, 
                                                                                          train_frac=0.7, 
                                                                                                min_cells_per_ident=min_cells)

    plot_validation_plots(validation_label_train_70t0vst1, valid_predlabels_train_70t0vst1, train_dict=t1_dict)

    test_labelst0vst1, test_predlabelst0vst1, test_prediction_t0vst1 = predict(train_anndata=adata_t1, 
                                                                     common_top_genes=model_t1.feature_names, 
                                                                     bst_model_train_full=model_t1, 
                                                                     test_anndata=adata_t0,
                                                                     train_obs_id=train_lab, 
                                                                     test_dict=t0_dict, 
                                                                     test_obs_id=test_lab)

    mappingconfmatt0vst1, mappingxtickst0vst1, mappingplott0vst1 = plot_mapping(test_labels=test_labelst0vst1, 
                 test_predlabels=test_predlabelst0vst1, 
                 test_dict=t0_dict, 
                 train_dict=t1_dict,
                     xaxislabel=x_lab, yaxislabel=y_lab,)
    
    test_items = test_labelst0vst1, test_predlabelst0vst1, test_prediction_t0vst1
    mapping_items = mappingconfmatt0vst1, mappingxtickst0vst1, mappingplott0vst1
    
    return test_items, mapping_items

def make_colors(PX_dict):
    PX_colors = []
    for i in PX_dict:
        for j in colors_subclass:
            if (j in i):
                PX_colors.append(colors_subclass[j])
    return PX_colors

def rem_fem(adata):
    for j in list(adata.obs.Type_nn_dists.values.categories):
        if ('Fem' in j):
            adata = adata[adata.obs['Type_nn_dists']!=j,:]
            adata = adata[adata.obs['Type_leiden']!=j,:]
    return adata

def list2dict(list_in):

    adata_dict = {}
    for num,i in enumerate(list_in):
        adata_dict[i] = num
        
    #adata_dict['Unassigned'] = num + 1
    
    return adata_dict

In [ ]:
test_items, mapping_items = pairwise_map(adata_t0=P12_FF_gaba, adata_t1=P22_gaba,test_lab='Type',
                                                                            train_lab='Type', 
                                                                          recomp_HVGs=True, union_hvgs=False,
                                         t0_dict=make_dict(P12_FF_gaba, 'Type'),
                                         t1_dict=make_dict(P22_gaba, 'Type'),
                                        min_cells=100, x_lab='P22 Sham', y_lab='P12_FF Sham')

test_labelsP12_FFvsP22, test_predlabelsP12_FFvsP22, test_prediction_P12_FFvsP22 = test_items
mappingconfmatP12_FFvsP22, mappingxticksP12_FFvsP22_sub, mappingplotP12_FFvsP22 = mapping_items

In [ ]:
test_labelsP12_FFvsP22, test_predlabelsP12_FFvsP22, test_prediction_P12_FFvsP22 = test_items
mappingconfmatP12_FFvsP22, mappingxticksP12_FFvsP22_sub, mappingplotP12_FFvsP22 = mapping_items

In [ ]:
a = make_dict(P22_gaba, 'Type')
P12_FF_gaba.obs['P22 Mapping Prob'] = np.max(test_prediction_P12_FFvsP22, axis=1)
P12_FF_gaba.obs['P22 Mapping Label'] = pd.Categorical(test_predlabelsP12_FFvsP22).rename_categories(list(a.keys()))

P12_FF_gaba.shape

type_cols = sn.color_palette('pastel').as_hex()+sn.color_palette('Set2').as_hex()+sn.color_palette('Set3').as_hex()+sn.color_palette('hls').as_hex()+sn.color_palette('husl').as_hex()
sc.set_figure_params(dpi_save=300)
sc.pl.umap(P12_FF_gaba, color=['Type',], legend_loc='on data', 
           legend_fontsize=10, palette=type_cols, legend_fontweight='semibold',s=30, 
           frameon=False,
           title= 'P22 1d WD: XXXX GABAergic Neurons',
           add_outline=True, save='P22_1-dWD_gaba')

P22_gaba.shape

type_cols = sn.color_palette('pastel').as_hex()+sn.color_palette('Set2').as_hex()+sn.color_palette('Set3').as_hex()+sn.color_palette('hls').as_hex()+sn.color_palette('husl').as_hex()
sc.set_figure_params(dpi_save=300)
sc.pl.umap(P22_gaba, color=['Type',], legend_loc='on data', 
           legend_fontsize=10, palette=type_cols, legend_fontweight='semibold',s=30, 
           frameon=False,
           title= 'P22: 4888 GABAergic Neurons',
           add_outline=True, save='P22_gaba')

In [ ]:
sc.pl.umap(P12_FF_gaba, color=['Type','P22 Mapping Prob','P22 Mapping Label', 'Baz1a',
                           'Trpc6', 'Igfn1'], legend_loc='on data', legend_fontsize=12)

In [ ]:
P22_gaba.obs.Subclass.values.categories

P22_gaba.obs['Sub'] = P22_gaba.obs['Subclass'].cat.rename_categories(['L2/3', 'L4', 'L4/5IT', 'L5NP', 
                                                                      'L5PT', 'L6CT', 'L6IT', 'L6b'])
P12_FF_gaba.obs['Sub'] = P12_FF_gaba.obs['Subclass'].cat.rename_categories(['L2/3', 'L4', 'L4/5IT', 'L5NP', 
                                                                      'L5PT', 'L6CT', 'L6IT', 'L6b'])

In [ ]:
c = 0
lims = [0.65, 0.35, 0.18, 0.43]
low_lims = [0, 0,0,0]
for i in ['Lamp5', 'Pvalb', 'Sst', 'Vip']:
    print(i)
    a=P22_gaba[P22_gaba.obs['Subclass'].str.startswith(i),:]
    b=P12_FF_gaba[P12_FF_gaba.obs['Subclass'].str.startswith(i),:]
    
    xx = a.obs['Type'].value_counts(normalize=True)
    yy = b.obs['P22 Mapping Label'].value_counts(normalize=True)
    
    #print(b.obs['P22 Mapping Label'].value_counts())
    
#     if (i=='L4'):
#         freq_scatter(x=xxx[0:4], y=yy[0:4],
#             x_lab='P22 Type Freqs', y_lab='P12_FF Types Mapping Freq', hue_='Types', unity_lim=lims[c], 
#                  low_lim=low_lims[c])
#         oh=xxx
#         my=yy
    
    freq_scatter(x=xx[xx.index.str.startswith(i)],
            y=yy[yy.index.str.startswith(i)],
            x_lab='P22 Type Freqs', y_lab='P12_FF Types Mapping Freq', hue_='Types', unity_lim=lims[c], 
                 low_lim=low_lims[c])
    plt.legend(ncol=1, bbox_to_anchor=(1,1))
    plt.show()
    c = c +1

In [ ]:
P22_gaba.obs['Type'].values.categories

In [ ]:
ord_gab = ['Pvalb_A', 'Pvalb_B', 'Pvalb_C',
       'Pvalb_D', 'Pvalb_E', 'Sst_A', 'Sst_B', 'Sst_C', 'Sst_D', 'Sst_E',
       'Sst_F', 'Sst_G', 'Sst_H', 'Sst_I', 'Vip_A', 'Vip_B', 'Vip_C', 'Vip_D',
       'Vip_E','Lamp5_A', 'Lamp5_B', 'Lamp5_C',]

In [ ]:
freq_scatter(x=P22_gaba.obs['Type'].value_counts(normalize=True),
            y= P12_FF_gaba.obs['P22 Mapping Label'].value_counts(normalize=True),
            x_lab='P22 Type Frequency', y_lab='10d AWD Type Frequency', hue_='Types', unity_lim=0.15, low_lim=0,
            hue_ord = ord_gab)
plt.savefig('figures/gaba_10dWD_freqs.pdf',  bbox_inches='tight')

# L2/3 PC analysis

## PCA on type markers

These objs grabbed from gluta sub-by-sub mapping section

In [ ]:
L23_FF = sc.read_h5ad('h5ads_cmprsd/P12_FF_L23_mappedP22.h5ad')
L23_P22 = sc.read_h5ad('h5ads_cmprsd/P22_L23.h5ad')

L23_FF.X = L23_FF.raw.X
L23_P22.X = L23_P22.raw.X

In [ ]:
sn.boxenplot(data=L23_P22.obs, x='Sample', y='total_counts')
plt.show()
sn.boxenplot(data=L23_P22.obs, x='Sample', y='n_genes_by_counts')

In [ ]:
sn.boxenplot(data=L23_FF.obs, x='Sample', y='total_counts')
plt.show()
sn.boxenplot(data=L23_FF.obs, x='Sample', y='n_genes_by_counts')

In [ ]:
L23_P22 = L23_P22[L23_P22.obs.Sample!='S3']

In [ ]:
L23_FF.obs['P22 Mapping Label'].value_counts()

In [ ]:
L23_P22_marks = []
for i in L23_P22.obs['Type'].values.categories:
    L23_P22_marks.append(DE(L23_P22,obs_id='Type', obs_id_test=i, ref='rest', 
                            pts_thresh=0.2, lf_thresh=0.6))

In [ ]:
L23_FF.X, L23_P22.X

In [ ]:
marks = list(L23_P22_marks[0].index)+list(L23_P22_marks[1].index)+list(L23_P22_marks[2].index)

In [ ]:
len(marks)

**Compute scores here now relative to all genes instead of before relative to only 489 markers**

In [ ]:
x=module_score(L23_FF, genes_use=L23_P22_marks[0].index, score_name='A Score', verbose = True)
x=module_score(L23_FF, genes_use=L23_P22_marks[1].index, score_name='B Score', verbose = True)
x=module_score(L23_FF, genes_use=L23_P22_marks[2].index, score_name='C Score', verbose = True)

x=module_score(L23_P22, genes_use=L23_P22_marks[0].index, score_name='A Score', verbose = True)
x=module_score(L23_P22, genes_use=L23_P22_marks[1].index, score_name='B Score', verbose = True)
x=module_score(L23_P22, genes_use=L23_P22_marks[2].index, score_name='C Score', verbose = True)

In [ ]:
'Baz1a' in marks

In [ ]:
L23_P22 = L23_P22[:,marks+['Baz1a']]
L23_FF = L23_FF[:,marks+['Baz1a']]

In [ ]:
L23_P22.var['highly_variable'] = L23_P22.shape[1]*[True]
L23_FF.var['highly_variable'] = L23_FF.shape[1]*[True]

In [ ]:
L23_FF.raw = L23_FF
sc.pp.scale(L23_FF, max_value=10) #scale
sc.tl.pca(L23_FF, svd_solver='arpack') #run PCA

In [ ]:
L23_P22.raw = L23_P22
sc.pp.scale(L23_P22, max_value=10) #scale
sc.tl.pca(L23_P22, svd_solver='arpack',) #run PCA

In [ ]:
sc.pl.pca_variance_ratio(L23_FF)

In [ ]:
sc.pl.pca_variance_ratio(L23_P22)

In [ ]:
sc.pl.pca(shuffle(L23_FF), color=['P22 Mapping Label','Sample'],  components=['1,2', '2,3', '3,4',], ncols=3,
         legend_loc='on data', add_outline=True, outline_width=(0.2,0.05), s=20, )
sc.pl.pca(shuffle(L23_P22), color=['Type', 'Sample'],  components=['1,2', '2,3', '3,4',], ncols=3,
         legend_loc='on data',add_outline=True,outline_width=(0.2,0.05), s=20)

In [ ]:
L23_P22.raw.X

In [ ]:
L23_FF.raw.X

In [ ]:
L23_FF.raw.X.A.shape

### Rotation

In [ ]:
import numpy as np

# Define your 100x2 matrix
points = L23_FF.obsm['X_pca']

# Extract the first two columns
points_2d = points[:, :2]

# Define the rotation matrix
theta = np.radians(-190)
rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])

# Define the reflection matrix
#reflection_matrix = np.array([[-1, 0], [0, 1]])

# Combine rotation and reflection into a single transformation matrix
#combined_matrix = np.dot(reflection_matrix, rotation_matrix)

# Apply combined transformation to the first two columns
transformed_points_2d = np.dot(points_2d, rotation_matrix.T)

# Replace the transformed columns back into the original matrix
transformed_points = np.copy(points)
transformed_points[:, :2] = transformed_points_2d


L23_FF.obsm['X_pca_rot'] = transformed_points

axes = [sc.pl.embedding(L23_FF, color=['P22 Mapping Label'],
         legend_loc='on data', add_outline=True, outline_width=(0.2,0.05), s=20,basis='X_pca_rot',
                     cmap='hot',title='P12 Cells: P22 Labels', show=False),
        sc.pl.embedding(shuffle(L23_FF), color=['P22 Mapping Label'],
         legend_loc='on data', add_outline=True, outline_width=(0.2,0.05), s=20, components=['2,3'],
                        basis='X_pca_rot',
                     cmap='hot',title='P12 Cells: P22 Labels', show=False),
        sc.pl.embedding(L23_P22, color=['Type'],
         legend_loc='on data', add_outline=True, outline_width=(0.2,0.05), s=20,basis='X_pca',
                     cmap='hot', title='P22 Cells: Mature Types', show=False),
       sc.pl.embedding(L23_P22, color=['Type'],
         legend_loc='on data', add_outline=True, outline_width=(0.2,0.05), s=20,basis='X_pca',
                     cmap='hot', title='P22 Cells: Mature Types',  components=['2,3'], show=False)]

for ax in axes:
    ax.set_xlim([-15,15])
    ax.set_ylim([-15,15])
    ax.set_xticks(np.arange(-15,16,5))
    ax.set_yticks(np.arange(-15,16,5))
    #ax.set_title('P12: Colored by P22 Mapping')
    ax.grid(True, linewidth=0.5)
#plt.tight_layout()

In [ ]:
#Given the results above, we can safely call X_pca with X_pca_rot.

L23_FF.obsm['X_pca'] = transformed_points

In [ ]:
points = L23_FF.varm['PCs']

# Extract the first two columns
points_2d = points[:, :2]

# Define the rotation matrix
theta = np.radians(-190)
rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])

# Define the reflection matrix
#reflection_matrix = np.array([[-1, 0], [0, 1]])

# Combine rotation and reflection into a single transformation matrix
#combined_matrix = np.dot(reflection_matrix, rotation_matrix)

# Apply combined transformation to the first two columns
transformed_points_2d = np.dot(points_2d, rotation_matrix.T)

# Replace the transformed columns back into the original matrix
transformed_points = np.copy(points)
transformed_points[:, :2] = transformed_points_2d


L23_FF.varm['PCs_rot'] = transformed_points

### Projection

In [ ]:
L23_FF.X

In [ ]:
#sc.pp.scale(L23_FF, max_value=10) #scale

In [ ]:
L23_FF.obsm['X_pca_P22'] = np.matmul(L23_FF.X, L23_P22.varm['PCs'][:,0:4])

In [ ]:
plt.scatter(L23_FF.obsm['X_pca_P22'][:,0], L23_FF.obsm['X_pca_P22'][:,1], s=3)
plt.xlim([-16,16])
plt.ylim([-15,15])
plt.title('P22 10d-WD')
plt.show()
plt.scatter(L23_P22.obsm['X_pca'][:,0], L23_P22.obsm['X_pca'][:,1], s=3)
plt.xlim([-16,16])
plt.ylim([-15,15])
plt.title('P22')
plt.show()

In [ ]:
L23_FF.obsm['X_pca_P22'].shape

In [ ]:
L23_FF.obs['PC1'] = L23_FF.obsm['X_pca_P22'][:,0]
L23_FF.obs['PC2'] = L23_FF.obsm['X_pca_P22'][:,1]

In [ ]:
L23_P22.obs['PC1'] = L23_P22.obsm['X_pca'][:,0]
L23_P22.obs['PC2'] = L23_P22.obsm['X_pca'][:,1]

In [ ]:
ax = sc.pl.scatter(adata=L23_FF,
              x='PC1', 
              y='PC2', color='P22 Mapping Label',legend_loc='on data',
                   show=False)
ax.set_xlim([-15,15])
ax.set_ylim([-15,15])

In [ ]:
ax = sc.pl.embedding(shuffle(L23_FF), color=['P22 Mapping Label'],
         legend_loc='on data', add_outline=True, outline_width=(0.2,0.05), s=20,basis='X_pca_rot',
                     palette=sn.color_palette('Set2').as_hex()[0:3],
              show=False)
ax.set_xlim([-15,15])
ax.set_ylim([-15,15])
ax.set_xticks(np.arange(-15,16,5))
ax.set_yticks(np.arange(-15,16,5))
ax.set_title('P22 10d-WD: Colored by P22 Mapping')
ax.set_xlabel('ref_P22_PC1')
ax.set_ylabel('ref_P22_PC2')

plt.grid(False)
plt.savefig('figures/Fig6E_P12types.pdf')

plt.show()

In [ ]:
ax = sc.pl.embedding(shuffle(L23_P22_sub), color=['Type'],
         legend_loc='on data', add_outline=True, outline_width=(0.2,0.05), s=20,basis='X_pca',
                     palette=sn.color_palette('Set2').as_hex()[0:3],
              show=False)
ax.set_xlim([-15,15])
ax.set_ylim([-15,15])
ax.set_xticks(np.arange(-15,16,5))
ax.set_yticks(np.arange(-15,16,5))
ax.set_title('P22: Colored by Type Label')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

plt.grid(False)
plt.savefig('figures/Fig6E_P22types.pdf')

plt.show()

In [ ]:
axes = [sc.pl.embedding(L23_FF, color=['P22 Mapping Label'],
         legend_loc='on data', add_outline=True, outline_width=(0.2,0.05), s=20,basis='X_pca_rot',
                     cmap='hot',title='P22 10d AWD: Colored by P22 Mapping',palette=sn.color_palette('Set2').as_hex()[0:3], show=False),
        sc.pl.embedding(shuffle(L23_FF), color=['P22 Mapping Label'],
         legend_loc='on data', add_outline=True, outline_width=(0.2,0.05), s=20, palette=sn.color_palette('Set2').as_hex()[0:3],components=['2,3'],
                        basis='X_pca_rot',
                     cmap='hot',title='P22 10d AWD: Colored by P22 Mapping', show=False),]

for ax in axes:
    ax.set_xlim([-15,15])
    ax.set_ylim([-15,15])
    ax.set_xticks(np.arange(-15,16,5))
    ax.set_yticks(np.arange(-15,16,5))
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    #ax.set_title('P12: Colored by P22 Mapping')
    ax.grid(False,)
#plt.tight_layout()

In [ ]:
axes = [sc.pl.embedding(L23_FF, color=['P22 Mapping Label'],
         legend_loc='on data', add_outline=True, outline_width=(0.2,0.05), s=20,basis='X_pca_rot',
                     cmap='hot',title='P22 10d-WD: Colored by P22 Mapping',palette=sn.color_palette('Set2').as_hex()[0:3], show=False),
        sc.pl.embedding(shuffle(L23_FF), color=['P22 Mapping Label'],
         legend_loc='on data', add_outline=True, outline_width=(0.2,0.05), s=20, palette=sn.color_palette('Set2').as_hex()[0:3],components=['2,3'],
                        basis='X_pca_rot',
                     cmap='hot',title='P22 10d-WD: Colored by P22 Mapping', show=False),
        sc.pl.embedding(L23_P22_sub, color=['Type'],
         legend_loc='on data', add_outline=True, outline_width=(0.2,0.05),palette=sn.color_palette('Set2').as_hex()[0:3], s=20,basis='X_pca',
                     cmap='hot', title='P22: Colored by Type Label', show=False),
       sc.pl.embedding(L23_P22_sub, color=['Type'],
         legend_loc='on data', add_outline=True, outline_width=(0.2,0.05), palette=sn.color_palette('Set2').as_hex()[0:3],s=20,basis='X_pca',
                     cmap='hot', title='P22: Colored by Type Label',  components=['2,3'], show=False)]

for ax in axes:
    ax.set_xlim([-15,15])
    ax.set_ylim([-15,15])
    ax.set_xticks(np.arange(-15,16,5))
    ax.set_yticks(np.arange(-15,16,5))
    #ax.set_title('P12: Colored by P22 Mapping')
    ax.grid(True, linewidth=0.5)
#plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5.5))
axs[0].scatter(L23_FF.obsm['X_pca_P22'][:,0], L23_FF.obsm['X_pca_P22'][:,1],
              s=40, color='grey')

sn.kdeplot(L23_FF.obsm['X_pca_P22'][:,0], L23_FF.obsm['X_pca_P22'][:,1],
             cmap="hot", shade=True, bw_adjust=0.7, alpha=0.7,ax=axs[0],)
axs[0].set_xlim([-20,20])
axs[0].set_ylim([-20,20])
axs[0].set_title('P12_FF')
axs[0].set_xlabel('ref_PC1')
axs[0].set_ylabel('ref_PC2')

axs[1].scatter(L23_P22.obsm['X_pca'][:,0], L23_P22.obsm['X_pca'][:,1],
              s=40, color='grey')
sn.kdeplot(L23_P22.obsm['X_pca'][:,0], L23_P22.obsm['X_pca'][:,1],
          cmap="hot", shade=True, bw_adjust=0.7, alpha=0.7, ax=axs[1])
axs[1].set_xlim([-20,20])
axs[1].set_ylim([-20,20])
axs[1].set_title('P22')
axs[1].set_xlabel('ref_PC1')
axs[1].set_ylabel('ref_PC2')
plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))
axs[0].scatter(L23_FF.obsm['X_pca_P22'][:,1], L23_FF.obsm['X_pca_P22'][:,2],
              s=40, color='grey')

sn.kdeplot(L23_FF.obsm['X_pca_P22'][:,1], L23_FF.obsm['X_pca_P22'][:,2],
             cmap="hot", shade=True, bw_adjust=0.7, alpha=0.7,ax=axs[0],)
axs[0].set_xlim([-16,16])
axs[0].set_ylim([-15,15])
axs[0].set_title('P12_FF')
axs[0].set_xlabel('ref_PC2')
axs[0].set_ylabel('ref_PC3')

axs[1].scatter(L23_P22.obsm['X_pca'][:,1], L23_P22.obsm['X_pca'][:,2],
              s=40, color='grey')
sn.kdeplot(L23_P22.obsm['X_pca'][:,1], L23_P22.obsm['X_pca'][:,2],
          cmap="hot", shade=True, bw_adjust=0.7, alpha=0.7, ax=axs[1])
axs[1].set_xlim([-16,16])
axs[1].set_ylim([-15,15])
axs[1].set_title('P22')
axs[1].set_xlabel('ref_PC2')
axs[1].set_ylabel('ref_PC3')

### ABC score

In [ ]:
L23_FF.X

In [ ]:
L23_FF.raw.X, L23_P22.raw.X

In [ ]:
axes = sc.pl.embedding(shuffle(L23_FF), color=['A Score', 'B Score', 'C Score'],
         legend_loc='on data', add_outline=True, outline_width=(0.1,0.05), s=30,basis='X_pca_rot',
                     cmap='hot',sort_order=False, vmin=0, vmax=1,
              show=False)

for ax in axes:
    ax.set_xlim([-15,15])
    ax.set_ylim([-15,15])
    ax.set_xticks(np.arange(-15,16,5))
    ax.set_yticks(np.arange(-15,16,5))
    #ax.set_title('P12: Colored by P22 Mapping')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.grid(False)
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
plt.tight_layout()

plt.savefig('figures/Fig6E_P12.pdf')

In [ ]:
L23_P22_sub = sc.pp.subsample(L23_P22, n_obs = L23_FF.shape[0], copy=True)

In [ ]:
axes = sc.pl.embedding(L23_P22_sub, color=['A Score', 'B Score', 'C Score'],
         legend_loc='on data', add_outline=True, outline_width=(0.1,0.05), s=30,basis='X_pca',
                     cmap='hot',sort_order=False, vmin=0, vmax=1,
              show=False)

for ax in axes:
    ax.set_xlim([-15,15])
    ax.set_ylim([-15,15])
    ax.set_xticks(np.arange(-15,16,5))
    ax.set_yticks(np.arange(-15,16,5))
    #ax.set_title('P12: Colored by P22 Mapping')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.grid(False)
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
plt.tight_layout()

plt.savefig('figures/Fig6E_P22.pdf')

### Save projected objects

In [ ]:
L23_FF.write_h5ad('h5ads_cmprsd/P12_FF_L23_mappedP22_proj.h5ad')
L23_P22.write_h5ad('h5ads_cmprsd/P22_L23_projFF.h5ad')

### PC feature plots

In [ ]:
#this contains 20k genes
adata = sc.read_h5ad('h5ads_cmprsd/P22_L23.h5ad')
adata = adata[adata.obs.Sample!='S3']
L23_P22_marks = []
for i in adata.obs['Type'].values.categories:
    L23_P22_marks.append(DE(adata,obs_id='Type', obs_id_test=i, ref='rest', 
                            pts_thresh=0.2, lf_thresh=0.6))

In [ ]:
#these only contain 489 ABC markers
L23_FF = sc.read_h5ad('h5ads_cmprsd/P12_FF_L23_mappedP22_proj.h5ad')
L23_P22 = sc.read_h5ad('h5ads_cmprsd/P22_L23_projFF.h5ad')

In [ ]:
L23_FF.var_names_make_unique()
L23_P22.var_names_make_unique()

In [ ]:
sc.pl.pca_variance_ratio(L23_FF, )
sc.pl.pca_variance_ratio(L23_P22,)

In [ ]:
plt.scatter(np.arange(30),100*L23_FF.uns['pca']['variance'][0:30]/sum(L23_FF.uns['pca']['variance'][0:30]),
           label='L2/3 P22 10d AWD')
plt.scatter(np.arange(30),100*L23_P22.uns['pca']['variance'][0:30]/sum(L23_P22.uns['pca']['variance'][0:30]),
           label='L2/3 P22')
plt.xlabel('PC Rank')
plt.legend()
plt.ylabel('Percent Variance Explained (%)')
plt.savefig('figures/varPCs_10d.pdf',  bbox_inches='tight')

plt.show()

In [ ]:

np.random.seed(0)  # For reproducibility
A = L23_P22.varm['PCs'][:,0:10]
B = L23_P22.varm['PCs'][:,0:10]

plt.figure(figsize=(8, 6))
sn.heatmap((A.T.dot(B)), annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt=".1f")
plt.title('Correlation Heatmap of PCs')
plt.show()

In [ ]:

np.random.seed(0)  # For reproducibility
A = L23_P22.varm['PCs'][:,0:4]
B = L23_FF.varm['PCs_rot'][:,0:4]

n, c = A.shape

corr_matrix = A.T.dot(B)

corr_df = pd.DataFrame(corr_matrix, index=[f'PC{i+1}' for i in range(c)], columns=[f'PC{i+1}' for i in range(c)])

# Plot the heatmap
plt.figure(figsize=(4, 3))
sn.heatmap(corr_df, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt=".2f")
plt.title('Correlation of PCs')
plt.xlabel('P22 10d AWD')
plt.ylabel('P22')
plt.savefig('figures/corrPCs_10d.pdf', bbox_inches='tight')
plt.show()


In [ ]:
A = L23_P22.varm['PCs'][:,1]
B = L23_FF.varm['PCs_rot'][:,1]

A.T.dot(B)

In [ ]:
def pca_heatmap_raw(adata, component, use_raw=None, layer=None, groupby='Type', frac_top=0.33):
    attr = 'varm'
    keys = 'PCs'
    scores = getattr(adata, attr)[keys][:, component]
    dd = pd.DataFrame(scores, index=adata.var_names)
    var_names_pos = dd.sort_values(0, ascending=False).index[:20]

    var_names_neg = dd.sort_values(0, ascending=True).index[:20]

    pd2 = pd.DataFrame(adata.obsm['X_pca'][:, component], index=adata.obs.index)

    bottom_cells = pd2.sort_values(0).index[:int(frac_top*L23_P12.shape[0])].tolist()
    top_cells = pd2.sort_values(0, ascending=False).index[:int(frac_top*L23_P12.shape[0])].tolist()
    adat_use = adata[top_cells+bottom_cells]
    sc.pl.heatmap(adat_use, list(var_names_pos) + list(var_names_neg), 
                        show_gene_labels=True,
                        swap_axes=True, cmap='hot', 
                        use_raw=False, layer=layer, figsize=(3,8), groupby=groupby,standard_scale='var')

In [ ]:
def annot_bar(adata, groupby,bar_position='bottom'):
    adata.obs['typecol'] = adata.obs[groupby].cat.rename_categories(sn.color_palette('Set2').as_hex()[0:3])

    import matplotlib.pyplot as plt
    from  matplotlib.cm import ScalarMappable
    from  matplotlib.colors import ListedColormap, BoundaryNorm
    import numpy as np

    vals = np.arange(adata.shape[0])
    cols = list(adata.obs['typecol'].values)
    bounds = np.append(vals, vals[-1] + 1)

    cmap = ListedColormap(cols)
    norm = BoundaryNorm(bounds, ncolors=len(cols))

    fig, ax = plt.subplots(figsize=(3, 1))
    fig.subplots_adjust(bottom=0.75)
    
    if bar_position == 'bottom':
        orientation = 'horizontal'
        pad = 0.1
        shrink = 1.0
    elif bar_position == 'top':
        orientation = 'horizontal'
        pad = 0.1
        shrink = 1.0
    elif bar_position == 'left':
        orientation = 'vertical'
        pad = 0.1
        shrink = 1.0
    elif bar_position == 'right':
        orientation = 'vertical'
        pad = 0.1
        shrink = 1.0
    else:
        raise ValueError("Invalid bar_position. Use 'bottom', 'top', 'left', or 'right'.")
    
    cb = fig.colorbar(ScalarMappable(norm=norm, cmap=cmap),
                 cax=ax, orientation='horizontal', label=groupby)
    cb.set_ticks([])
    #plt.show()

In [ ]:
def annot_bar_var(gene_list, groupby='Type'):
    
    
    categories = []
    for i in gene_list:
    
        
        if (i.split('-')[0] in L23_P22_marks[0].index): categories.append('A')
        elif (i.split('-')[0] in L23_P22_marks[1].index): categories.append('B')
        elif (i.split('-')[0] in L23_P22_marks[2].index): categories.append('C')
    
    
    a = pd.Categorical(categories)
    print(a)
    b = a.rename_categories(sn.color_palette('Set2').as_hex()[0:3])

    import matplotlib.pyplot as plt
    from  matplotlib.cm import ScalarMappable
    from  matplotlib.colors import ListedColormap, BoundaryNorm
    import numpy as np

    vals = np.arange(len(gene_list))
    cols = list(b)
    bounds = np.append(vals, vals[-1] + 1)

    cmap = ListedColormap(cols)
    norm = BoundaryNorm(bounds, ncolors=len(cols))

    fig, ax = plt.subplots(figsize=(0.2, 12))
    #fig.subplots_adjust(bottom=0.75)
    
    cb = fig.colorbar(ScalarMappable(norm=norm, cmap=cmap),
                 cax=ax, orientation='vertical',)
    cb.set_ticks([])
    #plt.show()

In [ ]:
def cumulative_count_shuff(data):
    
    cum_A = np.cumsum(data == 'L2/3_A')
    cum_B = np.cumsum(data == 'L2/3_B')
    cum_C = np.cumsum(data == 'L2/3_C')

    plt.figure(figsize=(4, 2))
    plt.plot(cum_A, label='L2/3_A', drawstyle='steps-post', color=sn.color_palette('Set2').as_hex()[0], linewidth=3)
    plt.plot(cum_B, label='L2/3_B', drawstyle='steps-post',color= sn.color_palette('Set2').as_hex()[1],linewidth=3)
    plt.plot(cum_C, label='L2/3_C', drawstyle='steps-post',color=sn.color_palette('Set2').as_hex()[2],linewidth=3)

    plt.xlabel('Shuffled')
    plt.ylabel('Cumulative Count')
    plt.title('Random Sequence of A,B,C')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
# Function to calculate the binned PDF for a category
def binned_pdf_category(category, data, bin_size=50):
    # Total number of items in the category
    total_count = np.sum(data == category)
    
    # Length of the data
    n = len(data)
    
    # Number of full bins
    num_bins = n // bin_size
    
    # Initialize the PDF array
    pdf = np.zeros(num_bins)
    
    for i in range(num_bins):
        bin_start = i * bin_size
        bin_end = bin_start + bin_size
        bin_count = np.sum(data[bin_start:bin_end] == category)
        pdf[i] = bin_count / total_count
    
    return pdf

def pdf_category(data,component, pc_vals,bin_size=50, ylab='Cells', title=''):
    # Calculate the binned PDF for each category
    binned_pdf_A = binned_pdf_category('L2/3_A', data,bin_size)
    binned_pdf_B = binned_pdf_category('L2/3_B', data,bin_size)
    binned_pdf_C = binned_pdf_category('L2/3_C', data, bin_size)

    num_bins = len(binned_pdf_A)
    #x_values = (np.arange(0, num_bins) + 0.5) * bin_size / len(data)
    
    x_values_frac = (np.arange(0, num_bins) + 0.5) * bin_size / len(data)

    x_values = []
    for i in x_values_frac:
        x_values.append(pc_vals[int(len(pc_vals)*i)])

    # Plot the binned PDF for each category
    plt.figure(figsize=(4, 2))
    plt.plot(x_values, binned_pdf_A, label='A',drawstyle='steps-mid', alpha=1, color=sn.color_palette('Set2').as_hex()[0], linewidth=2)
    plt.plot(x_values, binned_pdf_B, label='B',drawstyle='steps-mid', alpha=1,color=sn.color_palette('Set2').as_hex()[1], linewidth=2)
    plt.plot(x_values, binned_pdf_C, label='C',drawstyle='steps-mid', alpha=1, color=sn.color_palette('Set2').as_hex()[2], linewidth=2)

    # Add labels, title, and legend
    plt.xlabel('PC'+str(component+1)+ ' Value')
    plt.ylabel('Fraction of '+ylab)
    plt.title(title)
    plt.legend(bbox_to_anchor=(1,1))
    plt.grid(True)

    # Show plot
    plt.show()


In [ ]:
def cumulative_fraction_index(category, data):
    cum_count = np.cumsum(data == category)
    cum_fraction = cum_count / np.arange(1, len(data) + 1)
    return cum_fraction

def cum_frac_index(data,component):


    # Cumulative fractions for each category
    cum_frac_A = cumulative_fraction_index('L2/3_A', data)
    cum_frac_B = cumulative_fraction_index('L2/3_B', data)
    cum_frac_C = cumulative_fraction_index('L2/3_C', data)

    # Plot cumulative fractions
    plt.figure(figsize=(4, 2))
    plt.plot(cum_frac_A, label='L2/3_A', drawstyle='steps-post', color=sn.color_palette('Set2').as_hex()[0], linewidth=3)
    plt.plot(cum_frac_B, label='L2/3_B', drawstyle='steps-post',color= sn.color_palette('Set2').as_hex()[1],linewidth=3)
    plt.plot(cum_frac_C, label='L2/3_C', drawstyle='steps-post',color=sn.color_palette('Set2').as_hex()[2],linewidth=3)


    plt.xlabel('Position Along PC'+str(component+1))
    plt.ylabel('Cumulative Fraction (idx)')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    
    
def cumulative_fraction_size(category, data):

    cum_count = np.cumsum(data == category)
    total_count = np.sum(data == category)
    cum_fraction = cum_count / total_count
    return cum_fraction

def cum_frac_size(data,component):


    # Cumulative fractions for each category
    cum_frac_A = cumulative_fraction_size('L2/3_A', data)
    cum_frac_B = cumulative_fraction_size('L2/3_B', data)
    cum_frac_C = cumulative_fraction_size('L2/3_C', data)

    # Plot cumulative fractions
    plt.figure(figsize=(4, 2))
    plt.plot(cum_frac_A, label='L2/3_A', drawstyle='steps-post', color=sn.color_palette('Set2').as_hex()[0], linewidth=3)
    plt.plot(cum_frac_B, label='L2/3_B', drawstyle='steps-post',color= sn.color_palette('Set2').as_hex()[1],linewidth=3)
    plt.plot(cum_frac_C, label='L2/3_C', drawstyle='steps-post',color=sn.color_palette('Set2').as_hex()[2],linewidth=3)


    plt.xlabel('Position Along PC'+str(component+1))
    plt.ylabel('Cumulative Fraction (size)')
    plt.legend()
    plt.grid(True)
    plt.show()



In [ ]:
def cumulative_count(data,component):
    
    cum_A = np.cumsum(data == 'L2/3_A')
    cum_B = np.cumsum(data == 'L2/3_B')
    cum_C = np.cumsum(data == 'L2/3_C')

    plt.figure(figsize=(4, 2))
    plt.plot(cum_A, label='L2/3_A', drawstyle='steps-post', color=sn.color_palette('Set2').as_hex()[0], linewidth=3)
    plt.plot(cum_B, label='L2/3_B', drawstyle='steps-post',color= sn.color_palette('Set2').as_hex()[1],linewidth=3)
    plt.plot(cum_C, label='L2/3_C', drawstyle='steps-post',color=sn.color_palette('Set2').as_hex()[2],linewidth=3)

    plt.xlabel('Position Along PC'+str(component+1))
    plt.ylabel('Cumulative Count')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
def runs_chi(categories):
    import numpy as np
    import scipy.stats as stats
    from statsmodels.sandbox.stats.runs import runstest_1samp

    # Runs Test


    category_mapping = {'L2/3_A': 0, 'L2/3_B': 1, 'L2/3_C': 2}
    numerical_categories = pd.Series(categories).map(category_mapping).values

    # Runs Test
    _, p_value_runs = runstest_1samp(numerical_categories, correction=False)
    print(f'Runs Test p-value: {p_value_runs}')


    # Chi-Square Test
    # Dividing the list into 10 segments
    segments = np.array_split(categories, 50)

    # Counting occurrences in each segment
    observed_counts = np.array([[np.sum(segment == 'L2/3_A'),
                                 np.sum(segment == 'L2/3_B'),
                                 np.sum(segment == 'L2/3_C')] for segment in segments])

    # Expected counts assuming uniform distribution
    expected_counts = np.full((10, 3), len(categories) / 30)

    # Chi-square test
    chi2, p_value_chi2, _, _ = stats.chi2_contingency(observed_counts)
    print(f'Chi-Square Test p-value: {p_value_chi2}')


In [ ]:
def pca_heatmap(adata, component, use_raw=None, layer=None, groupby='Type', frac_top=0.33):
    attr = 'varm'
    keys = 'PCs'
    scores = getattr(adata, attr)[keys][:, component]
    dd = pd.DataFrame(scores, index=adata.var_names)
    var_names_pos = dd.sort_values(0, ascending=False).index[:20]

    var_names_neg = dd.sort_values(0, ascending=True).index[:20]

    pd2 = pd.DataFrame(adata.obsm['X_pca'][:, component], index=adata.obs.index)

    bottom_cells = pd2.sort_values(0).index[:int(frac_top*adata.shape[0])].tolist()
    top_cells = pd2.sort_values(0, ascending=False).index[:int(frac_top*adata.shape[0])].tolist()
    adat_use = adata[top_cells+bottom_cells]
    
    adat_use.obs['PC'+str(component+1)+' Score'] = list(itertools.chain.from_iterable(pd2.sort_values(0, ascending=False).values[:int(frac_top*adata.shape[0])].tolist()))+list(itertools.chain.from_iterable(pd2.sort_values(0).values[:int(frac_top*adata.shape[0])].tolist()))
    
    sc.pp.scale(adat_use)
    sc.pl.heatmap(adat_use, list(var_names_pos) + list(var_names_neg), 
                        show_gene_labels=True,
                        swap_axes=True, cmap='coolwarm', 
                        use_raw=False,vmin=-2,vmax=2, layer=layer, figsize=(3,8), 
                  groupby='PC'+str(component+1)+' Score', num_categories=1)
    annot_bar(adat_use, groupby)
    #print(len(bottom_cells), len(top_cells))
    del adata.raw
    del adat_use.raw
    adata.obs[groupby+'_plot'] = adata.shape[0]*['All Grey']
    adata.obs['PC'+str(component+1)+' Score'] = adata.shape[0]*[0]
    
    adat_use.obs[groupby+'_plot'] = adat_use.obs[groupby]
    adata_plot = adata.concatenate(adat_use, batch_categories=['Grey', 'Colored'])
    
    
    #adata.obs['PC '+str(component+1)] = adata.obsm['X_pca'][:, component]
    
    
    #print(pd2.sort_values(0).values[:int(frac_top*adata.shape[0])])
    adata_plot
    sc.pl.umap(adata_plot, color=['PC'+str(component+1)+' Score', groupby+'_plot'],
               s=30,#components=['1,2','2,3'],
              cmap='coolwarm', palette=sn.color_palette('Set2').as_hex()[0:3], sort_order=False)

In [ ]:
def autocorr(categories, component, shuffle=False):
    # Function to create binary sequence for a category
    def binary_sequence(data, category):
        return np.array([1 if item == category else 0 for item in data])

    # Function to compute autocorrelation
    def autocorrelation(sequence):
        n = len(sequence)
        mean = np.mean(sequence)
        var = np.var(sequence)
        autocorr = np.correlate(sequence - mean, sequence - mean, mode='full') / (var * n)
        return autocorr[n-1:]

    # Binary sequences for each category
    binary_A = binary_sequence(categories, 'L2/3_A')
    binary_B = binary_sequence(categories, 'L2/3_B')
    binary_C = binary_sequence(categories, 'L2/3_C')

    # Compute autocorrelation for each binary sequence
    autocorr_A = autocorrelation(binary_A)
    autocorr_B = autocorrelation(binary_B)
    autocorr_C = autocorrelation(binary_C)

    # Plot autocorrelation
    lags = np.arange(len(autocorr_A))

    plt.figure(figsize=(4, 2))
    plt.plot(lags, autocorr_A, label='L2/3_A',color=sn.color_palette('Set2').as_hex()[0],)
    plt.plot(lags, autocorr_B, label='L2/3_B',color=sn.color_palette('Set2').as_hex()[1],)
    plt.plot(lags, autocorr_C, label='L2/3_C',color=sn.color_palette('Set2').as_hex()[2])

    #plt.xlabel('Lag')
    plt.ylabel('Autocorrelation')
    
    if (not shuffle): 
        plt.xlabel('Lag along PC'+str(component+1))
    if (shuffle):
        plt.xlabel('Shuffled Sequence')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
def make_hmap(adata, genes, groupby,
             show_gene_labels=True, num_categories=1, layer=None, figsize=(3,8)):


    categories = []
    
    genes_loop = genes
    genes = []
    for i in genes_loop:
        
        if ('-' not in i):
            genes.append(i)
        
        if (i in L23_P22_marks[0].index): categories.append('A')
        elif (i in L23_P22_marks[1].index or i =='Baz1a'): categories.append('B')
        elif (i in L23_P22_marks[2].index): categories.append('C')
    
    print(len(genes), len(categories))
    # Create a DataFrame for gene categories
    gene_category_df = pd.DataFrame({
        'gene': genes,
        'Type': categories
    })

    # Set the gene names as the index
    gene_category_df.set_index('gene', inplace=True)

    # Create a custom colormap for the categories
    category_colors = {
        'A': sn.color_palette('Set2').as_hex()[0],
        'B': sn.color_palette('Set2').as_hex()[1],
        'C': sn.color_palette('Set2').as_hex()[2]
    }

    # Plot the heatmap
    a = sc.pl.heatmap(
        adata,
        var_names=genes,
        groupby=groupby,
        swap_axes=True,
        cmap='coolwarm',
        vmax=2,
        vmin=-2,show_gene_labels=show_gene_labels, num_categories=1,
                        use_raw=False, layer=layer, figsize=(3,8),
        show=False,)['heatmap_ax']

    # Set the y-tick labels with colors
    y_labels = a.get_yticklabels()

    for label in y_labels:
        gene = label.get_text()
        if gene in gene_category_df.index:
            category = gene_category_df.loc[gene, 'Type']
            label.set_color(category_colors[category])

    # Set the y-tick labels to be italic
    a.set_yticklabels(labels=genes, fontstyle='italic')

    plt.show()


In [ ]:
def pca_heatmap_all_genes_OLD(adata, component, use_raw=None, layer=None, groupby='Type'):
    #all cells, top X genes
    attr = 'varm'
    if (groupby!='Type'):keys = 'PCs_rot'
    else: keys = 'PCs'
    scores = getattr(adata, attr)[keys][:, component]
    dd = pd.DataFrame(scores, index=adata.var_names)
    var_names_pos = dd.sort_values(0, ascending=False).index
    median = int(adata.shape[1]/2)
    
    #print(median)
    #var_names_mid = dd.sort_values(0, ascending=False).index[median-10:median+10]
    #print(var_names_mid)
    
    var_names_neg = dd.sort_values(0, ascending=True).index

    pd2 = pd.DataFrame(adata.obsm['X_pca'][:, component], index=adata.obs.index)

    top_cells = pd2.sort_values(0, ascending=False).index.tolist()
    adat_use = adata[top_cells,:]
    
    adat_use.obs['PC'+str(component+1)+' Score'] = list(itertools.chain.from_iterable(pd2.sort_values(0, ascending=False).values.tolist()))#+list(itertools.chain.from_iterable(pd2.sort_values(0).values[:int(frac_top*adata.shape[0])].tolist()))
    
    sc.pp.scale(adat_use)
#     sc.pl.heatmap(adat_use, list(var_names_pos) + list(var_names_neg), 
#                         show_gene_labels=True, num_categories=1,
#                         swap_axes=True, cmap='coolwarm', 
#                         use_raw=False,vmin=-2,vmax=2, layer=layer, figsize=(3,8), #var_group_positions=((0,20), (20,40), (40,60)),var_group_labels=['Top', 'Middle', 'Bottom'],
#                   groupby='PC'+str(component+1)+' Score',)

    make_hmap(adata=adat_use, genes=list(var_names_pos), 
              groupby='PC'+str(component+1)+' Score', show_gene_labels=False)
    
    annot_bar_var(gene_list=list(var_names_pos))
    
    
    annot_bar(adat_use, groupby)
    #print(len(bottom_cells), len(top_cells))
    cumulative_count(np.array(adat_use.obs[groupby]), component)
    cum_frac_size(np.array(adat_use.obs[groupby]), component)
    cum_frac_index(np.array(adat_use.obs[groupby]), component)
    
    pdf_category(np.array(adat_use.obs[groupby]), component)
    
    autocorr(np.array(adat_use.obs[groupby]), component)
    runs_chi(np.array(adat_use.obs[groupby]))
    sc.pl.pca(adat_use, color=['PC'+str(component+1)+' Score', groupby],legend_loc='on data',legend_fontoutline=3,
               s=30,#components=['1,2','2,3'],
              cmap='coolwarm', vmin=-10, vmax=10,palette=sn.color_palette('Set2').as_hex()[0:3], sort_order=True)
    print('--------------------------------------------------------')
    #return np.array(adat_use.obs[groupby])
    #return dd.sort_values(0, ascending=False).index[:50], dd.sort_values(0, ascending=True).index[:50]
    return adat_use


In [ ]:
def pca_heatmap_all(adata, component, use_raw=None, layer=None, groupby='Type'):
    #all cells, top X genes
    attr = 'varm'
    if (groupby!='Type'):keys = 'PCs_rot'
    else: keys = 'PCs'
    scores = getattr(adata, attr)[keys][:, component]
    dd = pd.DataFrame(scores, index=adata.var_names)
    var_names_pos = dd.sort_values(0, ascending=False).index[:20]
    median = int(adata.shape[1]/2)
    
    #print(median)
    #var_names_mid = dd.sort_values(0, ascending=False).index[median-10:median+10]
    #print(var_names_mid)
    
    var_names_neg = dd.sort_values(0, ascending=True).index[:20]

    pd2 = pd.DataFrame(adata.obsm['X_pca'][:, component], index=adata.obs.index)

    top_cells = pd2.sort_values(0, ascending=False).index.tolist()
    adat_use = adata[top_cells,:]
    
    adat_use.obs['PC'+str(component+1)+' Score'] = list(itertools.chain.from_iterable(pd2.sort_values(0, ascending=False).values.tolist()))#+list(itertools.chain.from_iterable(pd2.sort_values(0).values[:int(frac_top*adata.shape[0])].tolist()))
    
    sc.pp.scale(adat_use)
#     sc.pl.heatmap(adat_use, list(var_names_pos) + list(var_names_neg), 
#                         show_gene_labels=True, num_categories=1,
#                         swap_axes=True, cmap='coolwarm', 
#                         use_raw=False,vmin=-2,vmax=2, layer=layer, figsize=(3,8), #var_group_positions=((0,20), (20,40), (40,60)),var_group_labels=['Top', 'Middle', 'Bottom'],
#                   groupby='PC'+str(component+1)+' Score',)

    make_hmap(adata=adat_use, genes=list(var_names_pos) + list(var_names_neg), 
              groupby='PC'+str(component+1)+' Score')
    
    
    annot_bar(adat_use, groupby)
    #print(len(bottom_cells), len(top_cells))
    cumulative_count(np.array(adat_use.obs[groupby]), component)
    
    autocorr(np.array(adat_use.obs[groupby]), component)
    runs_chi(np.array(adat_use.obs[groupby]))
    sc.pl.umap(adat_use, color=['PC'+str(component+1)+' Score', groupby],legend_loc='on data',legend_fontoutline=3,
               s=30,#components=['1,2','2,3'],
              cmap='coolwarm', vmin=-10, vmax=10,palette=sn.color_palette('Set2').as_hex()[0:3], sort_order=True)
    print('--------------------------------------------------------')
    #return np.array(adat_use.obs[groupby])
    return dd.sort_values(0, ascending=False).index[:50], dd.sort_values(0, ascending=True).index[:50]


In [ ]:
def pca_heatmap_marks(adata, component, use_raw=None, layer=None, groupby='Type'):
    attr = 'varm'
    if (groupby!='Type'):keys = 'PCs_rot'
    else: keys = 'PCs'
    print(keys)
    scores = getattr(adata, attr)[keys][:, component]
    dd = pd.DataFrame(scores, index=adata.var_names)
    var_names_pos = dd.sort_values(0, ascending=False).index[:20]
    median = int(adata.shape[1]/2)
    
    #print(median)
    var_names_mid = dd.sort_values(0, ascending=False).index[median-10:median+10]
    #print(var_names_mid)
    
    var_names_neg = dd.sort_values(0, ascending=True).index[:20]

    pd2 = pd.DataFrame(adata.obsm['X_pca'][:, component], index=adata.obs.index)

    top_cells = pd2.sort_values(0, ascending=True).index.tolist()
    adat_use = adata[top_cells,:]
    
    adat_use.obs['PC'+str(component+1)+' Score'] = list(itertools.chain.from_iterable(pd2.sort_values(0, ascending=True).values.tolist()))#+list(itertools.chain.from_iterable(pd2.sort_values(0).values[:int(frac_top*adata.shape[0])].tolist()))
    
    sc.pp.scale(adat_use)
    sc.pl.heatmap(adat_use, var_names=list(L23_P22_marks[0].index)+list(L23_P22_marks[1].index) + list(L23_P22_marks[2].index), 
                        show_gene_labels=False, num_categories=1,
                        swap_axes=True, cmap='coolwarm', 
                        use_raw=False,vmin=-2,vmax=2, layer=layer, figsize=(3,8), #var_group_positions=((0,183), (183,236), (236,501)),var_group_labels=['A', 'B', 'C'],
                  groupby='PC'+str(component+1)+' Score',)
    annot_bar(adat_use, groupby)
    #print(len(bottom_cells), len(top_cells))
    cumulative_count(np.array(adat_use.obs[groupby]), component)
    pdf_category(np.array(adat_use.obs[groupby]), component, pc_vals=adat_use.obs['PC'+str(component+1)+' Score'].values)

    
    autocorr(np.array(adat_use.obs[groupby]), component)
    runs_chi(np.array(adat_use.obs[groupby]))
    sc.pl.umap(adat_use, color=['PC'+str(component+1)+' Score', groupby],legend_loc='on data',legend_fontoutline=3,
               s=30,#components=['1,2','2,3'],
              cmap='coolwarm', vmin=-10, vmax=10,palette=sn.color_palette('Set2').as_hex()[0:3], sort_order=True)
    print('--------------------------------------------------------')
    return np.array(adat_use.obs[groupby])

In [ ]:
def pca_heatmap_all_genes(adata, component, use_raw=None, layer=None, groupby='Type'):
    #all cells, top X genes
    attr = 'varm'
    if (groupby!='Type'):keys = 'PCs_rot'
    else: keys = 'PCs'
    scores = getattr(adata, attr)[keys][:, component]
    dd = pd.DataFrame(scores, index=adata.var_names)
    var_names_pos = dd.sort_values(0, ascending=False).index
    median = int(adata.shape[1]/2)
    
    #print(median)
    #var_names_mid = dd.sort_values(0, ascending=False).index[median-10:median+10]
    #print(var_names_mid)
    
    var_names_neg = dd.sort_values(0, ascending=True).index

    pd2 = pd.DataFrame(adata.obsm['X_pca'][:, component], index=adata.obs.index)

    top_cells = pd2.sort_values(0, ascending=True).index.tolist()
    adat_use = adata[top_cells,:]
    
    adat_use.obs['PC'+str(component+1)+' Score'] = list(itertools.chain.from_iterable(pd2.sort_values(0, ascending=True).values.tolist()))#+list(itertools.chain.from_iterable(pd2.sort_values(0).values[:int(frac_top*adata.shape[0])].tolist()))
    
    sc.pp.scale(adat_use)
#     sc.pl.heatmap(adat_use, list(var_names_pos) + list(var_names_neg), 
#                         show_gene_labels=True, num_categories=1,
#                         swap_axes=True, cmap='coolwarm', 
#                         use_raw=False,vmin=-2,vmax=2, layer=layer, figsize=(3,8), #var_group_positions=((0,20), (20,40), (40,60)),var_group_labels=['Top', 'Middle', 'Bottom'],
#                   groupby='PC'+str(component+1)+' Score',)

    make_hmap(adata=adat_use, genes=list(var_names_pos), 
              groupby='PC'+str(component+1)+' Score', show_gene_labels=False)
    
    annot_bar_var(gene_list=list(var_names_pos))
    
    
    annot_bar(adat_use, groupby)
    #print(len(bottom_cells), len(top_cells))
    cumulative_count(np.array(adat_use.obs[groupby]), component)
    cum_frac_size(np.array(adat_use.obs[groupby]), component)
    cum_frac_index(np.array(adat_use.obs[groupby]), component)
    
    pdf_category(np.array(adat_use.obs[groupby]), component, pc_vals=adat_use.obs['PC'+str(component+1)+' Score'].values)
    
    autocorr(np.array(adat_use.obs[groupby]), component)
    runs_chi(np.array(adat_use.obs[groupby]))
    sc.pl.pca(adat_use, color=['PC'+str(component+1)+' Score', groupby],legend_loc='on data',legend_fontoutline=3,
               s=30,#components=['1,2','2,3'],
              cmap='coolwarm', vmin=-10, vmax=10,palette=sn.color_palette('Set2').as_hex()[0:3], sort_order=True)
    print('--------------------------------------------------------')
    #return np.array(adat_use.obs[groupby])
    #return dd.sort_values(0, ascending=False).index[:50], dd.sort_values(0, ascending=True).index[:50]
    return adat_use


In [ ]:
L23_P22_marks[2].shape

In [ ]:
236+265

In [ ]:
L23_FF.X

In [ ]:
def pca_heatmap_marks(adata, component, use_raw=None, layer=None, groupby='Type'):
    attr = 'varm'
    if (groupby!='Type'):keys = 'PCs_rot'
    else: keys = 'PCs'
    print(keys)
    scores = getattr(adata, attr)[keys][:, component]
    dd = pd.DataFrame(scores, index=adata.var_names)
    var_names_pos = dd.sort_values(0, ascending=False).index[:20]
    median = int(adata.shape[1]/2)
    
    #print(median)
    var_names_mid = dd.sort_values(0, ascending=False).index[median-10:median+10]
    #print(var_names_mid)
    
    var_names_neg = dd.sort_values(0, ascending=True).index[:20]

    pd2 = pd.DataFrame(adata.obsm['X_pca'][:, component], index=adata.obs.index)

    top_cells = pd2.sort_values(0, ascending=False).index.tolist()
    adat_use = adata[top_cells,:]
    
    adat_use.obs['PC'+str(component+1)+' Score'] = list(itertools.chain.from_iterable(pd2.sort_values(0, ascending=False).values.tolist()))#+list(itertools.chain.from_iterable(pd2.sort_values(0).values[:int(frac_top*adata.shape[0])].tolist()))
    
    sc.pp.scale(adat_use)
    sc.pl.heatmap(adat_use, var_names=list(L23_P22_marks[0].index)+list(L23_P22_marks[1].index) + list(L23_P22_marks[2].index), 
                        show_gene_labels=False, num_categories=1,
                        swap_axes=True, cmap='coolwarm', 
                        use_raw=False,vmin=-2,vmax=2, layer=layer, figsize=(3,12), #var_group_positions=((0,183), (183,236), (236,501)),var_group_labels=['A', 'B', 'C'],
                  groupby='PC'+str(component+1)+' Score',)
    annot_bar(adat_use, groupby)
    #print(len(bottom_cells), len(top_cells))
    cumulative_count(np.array(adat_use.obs[groupby]), component)
    
    autocorr(np.array(adat_use.obs[groupby]), component)
    runs_chi(np.array(adat_use.obs[groupby]))
    sc.pl.umap(adat_use, color=['PC'+str(component+1)+' Score', groupby],legend_loc='on data',legend_fontoutline=3,
               s=30,#components=['1,2','2,3'],
              cmap='coolwarm', vmin=-10, vmax=10,palette=sn.color_palette('Set2').as_hex()[0:3], sort_order=True)
    print('--------------------------------------------------------')
    return np.array(adat_use.obs[groupby])

In [ ]:
autocorr(np.array(shuffle(L23_FF.obs['P22 Mapping Label'])), component=None, shuffle=True)

In [ ]:
autocorr(np.array(shuffle(L23_P22.obs['Type'])), component=None, shuffle=True)

In [ ]:
cumulative_count_shuff(np.array(shuffle(L23_FF.obs['P22 Mapping Label'])))  # Replace with your list


In [ ]:
cumulative_count_shuff(np.array(shuffle(L23_P22.obs['Type'])))  # Replace with your list


In [ ]:
runs_chi(np.array(shuffle(L23_FF.obs['P22 Mapping Label'])))

In [ ]:
'Cdh12' in L23_P22_marks[2].index[0:22]

In [ ]:
'Cdh12' in L23_FF.var_names

In [ ]:
pca_heatmap_all_genes(L23_FF, 0, groupby='P22 Mapping Label')
pca_heatmap_all_genes(L23_FF, 1, groupby='P22 Mapping Label')

In [ ]:
pca_heatmap_all_genes(L23_P22, 0, groupby='Type')
pca_heatmap_all_genes(L23_P22, 1, groupby='Type')

pca_heatmap_marks(L23_FF, 0, groupby='P22 Mapping Label')
pca_heatmap_marks(L23_FF, 1, groupby='P22 Mapping Label')
pca_heatmap_marks(L23_FF, 2, groupby='P22 Mapping Label')
pca_heatmap_marks(L23_FF, 3, groupby='P22 Mapping Label')

In [ ]:
mark_labs = []
for i in L23_FF.var_names:
    if (i in L23_P22_marks[0].index): mark_labs.append('L2/3_A')
    elif (i in L23_P22_marks[1].index): mark_labs.append('L2/3_B')
    elif (i in L23_P22_marks[2].index): mark_labs.append('L2/3_C')
    else:
        print(i)
        mark_labs.append('N')

In [ ]:
len(L23_FF.var_names)

In [ ]:
len(mark_labs)

In [ ]:
L23_FF.var['Type'] = mark_labs
L23_P22.var['Type'] = mark_labs

L23_FF.var['PC1 P12'] = L23_FF.varm['PCs_rot'][:,0]
L23_FF.var['PC1 P22'] = L23_P22.varm['PCs'][:,0]

L23_FF.var['PC2 P12'] = L23_FF.varm['PCs_rot'][:,1]
L23_FF.var['PC2 P22'] = L23_P22.varm['PCs'][:,1]

In [ ]:
sn.distplot(L23_FF.var[L23_FF.var['Type']=='L2/3_B']['PC1 P12'])
sn.distplot(L23_FF.var[L23_FF.var['Type']=='L2/3_B']['PC1 P22'])

In [ ]:
def pdf_category_genes(data,component, ylim, bin_size=50, ylab='Cells', title=''):
    # Calculate the binned PDF for each category
    binned_pdf_A = binned_pdf_category('L2/3_A', data,bin_size)
    binned_pdf_B = binned_pdf_category('L2/3_B', data,bin_size)
    binned_pdf_C = binned_pdf_category('L2/3_C', data, bin_size)

    num_bins = len(binned_pdf_A)
    x_values = (np.arange(0, num_bins) + 0.5) * bin_size / len(data)

    
    # Plot the binned PDF for each category
    plt.figure(figsize=(4, 2))
    plt.plot(x_values, binned_pdf_A, label='A',drawstyle='steps-mid', alpha=1, color=sn.color_palette('Set2').as_hex()[0], linewidth=2)
    plt.plot(x_values, binned_pdf_B, label='B',drawstyle='steps-mid', alpha=1,color=sn.color_palette('Set2').as_hex()[1], linewidth=2)
    plt.plot(x_values, binned_pdf_C, label='C',drawstyle='steps-mid', alpha=1, color=sn.color_palette('Set2').as_hex()[2], linewidth=2)

    # Add labels, title, and legend
    plt.xlabel('Fractional Position Along PC'+str(component+1))
    plt.ylabel('Fraction of '+ylab)
    plt.title(title)
    plt.ylim(ylim)
    plt.legend()
    plt.grid(True)

    # Show plot
    plt.show()
    return binned_pdf_A,binned_pdf_B,binned_pdf_C

def diff(first, second, title):
    
    A = np.abs(first[0]-second[0])
    B = np.abs(first[1]-second[1])  
    C = np.abs(first[2]-second[2]) 

    plt.figure(figsize=(4,2))
    plt.plot(np.arange(len(A))/len(A), A, label='A', color=sn.color_palette('Set2').as_hex()[0])
    plt.plot(np.arange(len(A))/len(A), B, label='B',color=sn.color_palette('Set2').as_hex()[1])
    plt.plot(np.arange(len(A))/len(A), C, label='C',color=sn.color_palette('Set2').as_hex()[2])

    plt.xlabel('Position Along PC1')
    plt.ylabel('Absolute Difference')
    plt.grid(False)
    plt.title(title)
    plt.legend()

In [ ]:
FF_PC1 = pdf_category_genes(np.array(L23_FF.var.sort_values(by='PC1 P12', ascending=True)['Type']), 
             component=0,bin_size=10, ylab='Genes', title='P22 10d-WD', ylim=[0,0.2])
P22_PC1 = pdf_category_genes(np.array(L23_FF.var.sort_values(by='PC1 P22', ascending=True)['Type']), 
                             component=0,bin_size=10,
             ylab='Genes', title='P22', ylim=[0,0.2])


In [ ]:
FF_PC2 = pdf_category_genes(np.array(L23_FF.var.sort_values(by='PC2 P12', ascending=True)['Type']), component=1,
                            bin_size=10,
            ylab='Genes', title='P22 10d-WD',ylim=[0,0.22])
P22_PC2 = pdf_category_genes(np.array(L23_FF.var.sort_values(by='PC2 P22', ascending=True)['Type']), component=1,bin_size=10,
            ylab='Genes', title='P22',ylim=[0,0.22])


In [ ]:
from scipy.spatial.distance import jensenshannon

def jj(first, second, title):
    
    A = jensenshannon(first[0], second[0])**2
    B = jensenshannon(first[1], second[1])**2  # The function returns the square root of JS divergence
    C = jensenshannon(first[2], second[2])**2  # The function returns the square root of JS divergence

    plt.figure(figsize=(2.5,3))
    plt.bar(['A','B','C'], [A,B,C], color='#D294E5', edgecolor='black')
    plt.xlabel('L2/3 Type')
    plt.ylabel('Jenson-Shannon Divergence')
    plt.grid(False)
    plt.title(title)
    

In [ ]:
jj(FF_PC1, P22_PC1, 'PC1')
diff(FF_PC1, P22_PC1, 'PC1')

In [ ]:
jj(FF_PC2, P22_PC2, 'PC2')
diff(FF_PC2, P22_PC2, 'PC2')

In [ ]:
sn.scatterplot(data=L23_FF.var, x='PC1 P12', y='PC1 P22', hue='Type')
plt.xlim([-0.17,0.17])
plt.ylim([-0.17,0.17])
plt.plot(np.linspace(-0.2,0.2,10),np.linspace(-0.2,0.2,10), color='black')
plt.show()

sn.scatterplot(data=L23_FF.var, x='PC2 P12', y='PC2 P22', hue='Type')
plt.xlim([-0.17,0.17])
plt.ylim([-0.17,0.17])
plt.plot(np.linspace(-0.2,0.2,10),np.linspace(-0.2,0.2,10), color='black')

In [ ]:
category_colors = {
    'Category1': 'blue',
    'Category2': 'green',
    'Category3': 'red'
}

In [ ]:
p12_pc1p, p12_pc1n = pca_heatmap_all(L23_FF, 0, groupby='P22 Mapping Label')
p12_pc2p, p12_pc2n = pca_heatmap_all(L23_FF, 1, groupby='P22 Mapping Label')
# pca_heatmap_all(L23_FF, 2, groupby='P22 Mapping Label')
# pca_heatmap_all(L23_FF, 3, groupby='P22 Mapping Label')


In [ ]:
runs_chi(categories)

In [ ]:
pca_heatmap(L23_FF, 0, groupby='P22 Mapping Label', frac_top=0.2)
pca_heatmap(L23_FF, 1,groupby='P22 Mapping Label',frac_top=0.2)
pca_heatmap(L23_FF, 2,groupby='P22 Mapping Label',frac_top=0.2)
pca_heatmap(L23_FF, 3,groupby='P22 Mapping Label',frac_top=0.2)

In [ ]:
pca_heatmap_raw(L23_FF, 0, groupby='P22 Mapping Label', frac_top=0.33)
pca_heatmap_raw(L23_FF, 1,groupby='P22 Mapping Label',frac_top=0.33)
pca_heatmap_raw(L23_FF, 2,groupby='P22 Mapping Label',frac_top=0.33)
pca_heatmap_raw(L23_FF, 3,groupby='P22 Mapping Label',frac_top=0.33)

In [ ]:
sc.pl.umap(L23_FF, color=['P22 Mapping Label'],title='P12 Cells: P22 Mapping Label', 
               s=30,palette=sn.color_palette('Set2').as_hex()[0:3])
sc.pl.umap(L23_P22, color=['Type'],title='P22 Cells: Type', 
               s=30,palette=sn.color_palette('Set2').as_hex()[0:3])

In [ ]:
pca_heatmap(L23_FF, 0,)
pca_heatmap(L23_FF, 1)
pca_heatmap(L23_FF, 2)
pca_heatmap(L23_FF, 3)

In [ ]:
sc.pl.pca(L23_FF, color=['P22 Mapping Label','P22 Mapping Prob'])

In [ ]:
pca_heatmap_marks(L23_P22, 0)
pca_heatmap_marks(L23_P22, 1)
pca_heatmap_marks(L23_P22, 2)
pca_heatmap_marks(L23_P22, 3)


In [ ]:
loadings1.shape

In [ ]:
loadings2.shape

In [ ]:
L23_P22_marks[0].shape, L23_P22_marks[1].shape, L23_P22_marks[2].shape

In [ ]:
loadings1 = L23_FF.varm['PCs_rot'][:,0][:183]  # Loadings for PC1 in dataset1
loadings2 = L23_P22.varm['PCs'][:,0][:183]  # Loadings for PC1 in dataset2

# Create a DataFrame to compare loadings
features = L23_P22.var_names[:183]  # List of feature names
comparison_df = pd.DataFrame({
    'gene': features,
    'P22 10d-WD': loadings1,
    'P22': loadings2,
    'P22 - P12': loadings2 - loadings1,
    'P22 / P12': loadings2 / (loadings1+0.01)
})

# Visualize the loadings
plt.figure(figsize=(50, 6))
comparison_df.plot(x='gene', y='P22 - P12', kind='bar', ax=plt.gca())
plt.title('Comparison of PC1 Loadings')
plt.ylabel('Loading Ratio')
plt.show()


In [ ]:
!pip install POT

In [ ]:
loadings1 = L23_FF.varm['PCs_rot'][:,0]  # Loadings for PC1 in dataset1
loadings2 = L23_P22.varm['PCs'][:,0]  # Loadings for PC1 in dataset2

# Create a DataFrame to compare loadings
features = L23_P22.var_names  # List of feature names
comparison_df = pd.DataFrame({
    'Feature': features,
    'Loadings_Dataset1': loadings1,
    'Loadings_Dataset2': loadings2,
    'Difference': loadings1 - loadings2,
    'Ratio': loadings1 / loadings2
})

# Visualize the loadings
plt.figure(figsize=(50, 6))
comparison_df.plot(x='Feature', y='Ratio', kind='bar', ax=plt.gca())
plt.title('Comparison of PC1 Loadings')
plt.ylabel('Loading Value')
plt.show()


In [ ]:
p22pc1p , p22pc1n = pca_heatmap_all(L23_P22, 0)
p22pc2p , p22pc2n = pca_heatmap_all(L23_P22, 1)
# pca_heatmap_all(L23_P22, 2)
# pca_heatmap_all(L23_P22, 3)

In [ ]:
def jaccard_index(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    
    if not union:
        return 0.0
    
    return len(intersection) / len(union)

# Example usage:
list1 = [1, 2, 3, 4]
list2 = [3, 4, 5, 6]

print(jaccard_index(list1, list2))


In [ ]:
jaccard_index(p12_pc1p, p22pc1p), jaccard_index(p12_pc2p, p22pc2p)

In [ ]:
jaccard_index(p12_pc1n, p22pc1n), jaccard_index(p12_pc2n, p22pc2n)

In [ ]:
pca_heatmap(L23_P22, 0,frac_top=0.2)
pca_heatmap(L23_P22, 1,frac_top=0.2)
pca_heatmap(L23_P22, 2,frac_top=0.2)
pca_heatmap(L23_P22, 3,frac_top=0.2)

In [ ]:
pca_heatmap_raw(L23_P22, 0, groupby='Type', frac_top=0.33)
pca_heatmap_raw(L23_P22, 1,groupby='Type',frac_top=0.33)
pca_heatmap_raw(L23_P22, 2,groupby='Type',frac_top=0.33)
pca_heatmap_raw(L23_P22, 3,groupby='Type',frac_top=0.33)

In [ ]:
pd2 = pd.DataFrame(L23_FF.obsm['X_pca'][:, 0], index=L23_FF.obs.index)
top_cells_12 = pd2.sort_values(0, ascending=False).index.tolist()

In [ ]:
pd2 = pd.DataFrame(L23_P22.obsm['X_pca'][:, 0], index=L23_P22.obs.index)
top_cells_22 = pd2.sort_values(0, ascending=False).index.tolist()

In [ ]:
L23.obs

In [ ]:
pd2.sort_values(0, ascending=False)

In [ ]:
L23.obs.index = L23.obs.index.str.replace('-P12','')
L23.obs.index = L23.obs.index.str.replace('-P22','')

In [ ]:
L23_ord = L23[top_cells_12+top_cells_22,:].copy()
L23_ord.obs['Order'] = np.arange(L23_ord.shape[0])

In [ ]:
L23_ord.obs['Order']

In [ ]:
sc.pl.heatmap(L23_ord, var_names=list(L23_P22_marks[0].index)+list(L23_P22_marks[1].index)+list(L23_P22_marks[2].index), 
              groupby='Type_Age', 
              swap_axes=True,use_raw=False, vmin=-2, vmax=2, cmap='coolwarm', )

### Sample markers Fig 3

In [ ]:
adata = L23_P22.copy()

In [ ]:
L23_P22.X, L23_FF.X

#### ABC scores

In [ ]:
def plt_score_idx(datasets, genes, title_colors, component=0, bin_size=75, nrows=1, ncols=1):
    # Create a figure with the specified number of rows and columns for subplots
    fig, axes = plt.subplots(nrows, ncols, figsize=(4 * ncols, 2.5 * nrows), sharey=False, sharex=True)
    
    # Flatten axes array for easy iteration if it's multi-dimensional
    axes = axes.flatten()

    c = 0
    for ax, gene in zip(axes, genes):
        for adata, age, color in datasets:
            pd2 = pd.DataFrame(adata.obsm['X_pca'][:, component], index=adata.obs.index)
            top_cells = pd2.sort_values(0, ascending=True).index.tolist()
            adat_use = adata[top_cells, :]

            positions = np.arange(adat_use.shape[0])
            expression_values  = adat_use.obs[gene].values
            
            # Number of full bins
            num_bins = len(positions) // bin_size

            # Initialize arrays to store binned positions and expression values
            binned_positions = np.zeros(num_bins)
            binned_expression = np.zeros(num_bins)

            for i in range(num_bins):
                bin_start = i * bin_size
                bin_end = bin_start + bin_size
                binned_positions[i] = (bin_start + bin_end) / 2  # Middle of the bin
                binned_expression[i] = np.mean(expression_values[bin_start:bin_end])

            # Normalize the x-axis values to range from 0 to 1
            normalized_positions = binned_positions / len(positions)
                

            # Plot the binned gene expression values
            ax.plot(normalized_positions, binned_expression, drawstyle='steps-mid', linewidth=2, label=age, color=color)

        # Add labels, title, and legend for each subplot
        ax.set_title(gene, color=title_colors[c], style='italic')
        ax.legend()
        ax.grid(True)
        c += 1

    # Hide unused subplots
    for i in range(len(genes), len(axes)):
        fig.delaxes(axes[i])

    # Adjust layout to prevent overlap
    plt.tight_layout()
    
    # Show plot
    plt.show()

In [ ]:
from scipy.stats import ks_2samp, ks_1samp

In [ ]:
ks_1samp()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

def plt_score_idx_with_ttest(datasets, genes, title_colors, save, component=0, bin_size=75, nrows=1, ncols=1):
    # Create a figure with the specified number of rows and columns for subplots
    fig, axes = plt.subplots(nrows, ncols, figsize=(4 * ncols, 2.5 * nrows), sharey=False, sharex=True)
    
    # Flatten axes array for easy iteration if it's multi-dimensional
    axes = axes.flatten()

    c = 0
    for ax, gene in zip(axes, genes):
        binned_expressions = []
        for adata, age, color in datasets:
            pd2 = pd.DataFrame(adata.obsm['X_pca'][:, component], index=adata.obs.index)
            top_cells = pd2.sort_values(0, ascending=True).index.tolist()
            adat_use = adata[top_cells, :]

            positions = np.arange(adat_use.shape[0])
            expression_values  = adat_use.obs[gene].values
            
            
            # Number of full bins
            num_bins = len(positions) // bin_size

            # Initialize arrays to store binned positions and expression values
            binned_positions = np.zeros(num_bins)
            binned_expression = np.zeros(num_bins)

            for i in range(num_bins):
                bin_start = i * bin_size
                bin_end = bin_start + bin_size
                binned_positions[i] = (bin_start + bin_end) / 2  # Middle of the bin
                binned_expression[i] = np.mean(expression_values[bin_start:bin_end])

            # Normalize the x-axis values to range from 0 to 1
            normalized_positions = binned_positions / len(positions)
            
            # Plot the binned gene expression values
            ax.plot(normalized_positions, binned_expression, drawstyle='steps-mid', linewidth=2, label=age, color=color)
                    
            # Collect binned expressions for t-test
            binned_expressions.append(binned_expression)

        # Perform t-test between the two sets of binned expressions
        if len(binned_expressions) == 2:
            t_stat, p_val = ks_2samp(binned_expressions[0], binned_expressions[1],)
            # Include p-value in the title
            ax.set_title(f'{gene}\n(p = {p_val:.2e})', color=title_colors[c], style='italic')
        else:
            ax.set_title(gene, color=title_colors[c], style='italic')

        # Add labels, title, and legend for each subplot
        ax.legend()
        ax.grid(True)
        c += 1

    # Hide unused subplots
    for i in range(len(genes), len(axes)):
        fig.delaxes(axes[i])

    # Adjust layout to prevent overlap
    plt.tight_layout()
    
    # Show plot
    plt.savefig(save)
    plt.show()

In [ ]:
def plt_gene_idx(datasets, genes, title_colors, save, component=0, bin_size=75, nrows=1, ncols=1):
    # Create a figure with the specified number of rows and columns for subplots
    fig, axes = plt.subplots(nrows, ncols, figsize=(4 * ncols, 2.7 * nrows), sharey=False)
    
    # Flatten axes array for easy iteration if it's multi-dimensional
    axes = axes.flatten()

    c = 0
    for ax, gene in zip(axes, genes):
        binned_expressions = []
        for adata, age, color in datasets:
            pd2 = pd.DataFrame(adata.obsm['X_pca'][:, component], index=adata.obs.index)
            top_cells = pd2.sort_values(0, ascending=True).index.tolist()
            adat_use = adata[top_cells, :]

            positions = np.arange(adat_use.shape[0])
            expression_values = adat_use.X[:, adat_use.var_names.get_loc(gene)].A  # values

            # Number of full bins
            num_bins = len(positions) // bin_size

            # Initialize arrays to store binned positions and expression values
            binned_positions = np.zeros(num_bins)
            binned_expression = np.zeros(num_bins)

            for i in range(num_bins):
                bin_start = i * bin_size
                bin_end = bin_start + bin_size
                binned_positions[i] = (bin_start + bin_end) / 2  # Middle of the bin
                binned_expression[i] = np.mean(expression_values[bin_start:bin_end])

            # Normalize the x-axis values to range from 0 to 1
            normalized_positions = binned_positions / len(positions)

            # Plot the binned gene expression values
            ax.plot(normalized_positions, binned_expression, drawstyle='steps-mid', linewidth=2, label=age, color=color)
            
            # Collect binned expressions for t-test
            binned_expressions.append(binned_expression)
            #print(len(binned_positions), len(binned_expression))

         # Perform t-test between the two sets of binned expressions
#         if len(binned_expressions) == 2:
#             t_stat, p_val = ttest_ind(binned_expressions[0], binned_expressions[1], equal_var=False)
#             # Include p-value in the title
#             gene_bold_italic = f'\\textbf{{\\textit{{{gene}}}}}'
#             ax.set_title(f'{gene_bold_italic}\n(p = {p_val:.2e})', color=title_colors[c], style='italic', usetex=True)
#             #ax.set_title(f'{gene}\n(p = {p_val:.2e})', color=title_colors[c], style='italic')
#         else:
#             ax.set_title(gene, color=title_colors[c], style='italic', fontweight='bold')

        # Add labels, title, and legend for each subplot
        ax.set_title(gene, color=title_colors[c], style='italic', fontweight='bold')
        ax.legend()
        ax.grid(True)
        c += 1

    # Hide unused subplots
    for i in range(len(genes), len(axes)):
        fig.delaxes(axes[i])

    # Adjust layout to prevent overlap
    plt.tight_layout()
    
    # Show plot
    plt.savefig(save)
    plt.show()

In [ ]:
L23_FF.X = L23_FF.raw.X
L23_P22.X = L23_P22.raw.X

In [ ]:
# Example usage:
datasets = [
    (L23_FF, 'P22 10d AWD', '#e78ac3'),
    (L23_P22, 'P22', '#a6d854')
]

genes = ['A Score', 'B Score', 'C Score']  # Add your genes here

plt_score_idx_with_ttest(datasets, genes, title_colors=sn.color_palette('Set2').as_hex()[0:3], nrows=3, ncols=1,
                        save='figures/FigS8A.pdf')

Re-ran the PCA on Type markers 6.1 section, adding Baz1a to marks

In [ ]:
# Example usage:
datasets = [
    (L23_FF, 'P22 10d AWD', '#e78ac3'),
    (L23_P22, 'P22', '#a6d854')
]

genes = ['Cdh13', 'Trpc6', 'Chrm2'] +['Adamts2', 'Bdnf', 'Cdh12']+ ['Sorcs3', 'Igfn1','Baz1a',] 

plt_gene_idx(datasets, genes, title_colors=sn.color_palette('Set2').as_hex()[0:3]+sn.color_palette('Set2').as_hex()[0:3]+len(genes)*[sn.color_palette('Set2').as_hex()[1]], 
             nrows=3, ncols=3, save='figures/Fig6F.pdf')

In [ ]:
# Example usage:
datasets = [
    (L23_FF, 'P22 10d-WD', '#e78ac3'),
    (L23_P22, 'P22', '#a6d854')
]

genes = ['Cdh13', 'Trpc6', 'Chrm2'] +['Adamts2', 'Bdnf', 'Cdh12']+ ['Sorcs3', 'Igfn1','Baz1a',] 

plt_gene_idx(datasets, genes, title_colors=sn.color_palette('Set2').as_hex()[0:3]+sn.color_palette('Set2').as_hex()[0:3]+len(genes)*[sn.color_palette('Set2').as_hex()[1]], 
             nrows=3, ncols=3)

In [ ]:
L23_FF.obs.Sample

In [ ]:
# Example usage:
datasets = [
    (L23_FF, 'P22 10d-WD', '#e78ac3'),
    (L23_P22, 'P22', '#a6d854')
]

genes = list(L23_P22_marks[0].index[0:30]) +list(L23_P22_marks[1].index[0:30])+ list(L23_P22_marks[2].index[0:30])

plt_gene_idx(datasets, genes, title_colors=30*[sn.color_palette('Set2').as_hex()[0]]+30*[sn.color_palette('Set2').as_hex()[1]]+30*[sn.color_palette('Set2').as_hex()[2]], 
             nrows=10, ncols=9, bin_size=75)

#### TFs

In [ ]:
#
path_dbs = 'Final_Gene_Lists/'
csms = list(pd.read_csv(path_dbs+'csm_final.csv')['gene'])
ics = list(pd.read_csv(path_dbs+'ion_final.csv')['gene'])
nps = list(pd.read_csv(path_dbs+'Neuropeptides.csv', header=None, names=['Genes'])['Genes'])
ntr = list(pd.read_csv(path_dbs+'ntr_final.csv')['gene'])
gpcrs = list(pd.read_excel('MouseHumanRatGPCRs.xlsx')['Gene Symbol (Mouse)'].values)

L23_genes = pd.read_excel('L23_56.xlsx', index_col=0)
L23_genes = list(L23_genes.index)+['Tnc']

tfs = pd.read_table('TFs_mm.txt', header=0)
tfs = list(tfs['Bcl6b'].values)+['Bcl6b']

house = list(pd.read_table('Bulk Cluster Ubiquitous.txt',).transpose()[0:3000].index)

In [ ]:
'Cpne8' in csms

In [ ]:
datasets = [
    (L23_FF, 'P22 10d-WD', '#e78ac3'),
    (L23_P22, 'P22', '#a6d854')
]

genes = list(set(L23_P22_marks[0].index).intersection(set(tfs)))  # Add your genes here

plt_gene_idx(datasets, genes,title_colors=len(genes)*[sn.color_palette('Set2').as_hex()[0]], 
             nrows=4, ncols=3)

In [ ]:
datasets = [
    (L23_FF, 'P22 10d-WD', '#e78ac3'),
    (L23_P22, 'P22', '#a6d854')
]

genes = list(set(L23_P22_marks[1].index).intersection(set(tfs)))  # Add your genes here

plt_gene_idx(datasets, genes, title_colors=len(genes)*[sn.color_palette('Set2').as_hex()[1]],ncols=2, nrows=3)

In [ ]:
# Example usage:
datasets = [
    (L23_FF, 'P22 10d-WD', '#e78ac3'),
    (L23_P22, 'P22', '#a6d854')
]

genes = list(set(L23_P22_marks[2].index).intersection(set(tfs)))  # Add your genes here

plt_gene_idx(datasets, genes, title_colors=len(genes)*[sn.color_palette('Set2').as_hex()[2]],nrows=7, ncols=3)

#### ICs

In [ ]:
path_dbs = 'Final_Gene_Lists/'

ics = list(pd.read_csv(path_dbs+'ion_final.csv')['gene'])


In [ ]:
datasets = [
    (L23_FF, 'P22 10d-WD', '#e78ac3'),
    (L23_P22, 'P22', '#a6d854')
]

genes = list(set(L23_P22_marks[0].index).intersection(set(ics)))  # Add your genes here

plt_gene_idx(datasets, genes, title_colors=len(genes)*[sn.color_palette('Set2').as_hex()[0]],nrows=2, ncols=2)

In [ ]:
datasets = [
    (L23_FF, 'P22 10d-WD', '#e78ac3'),
    (L23_P22, 'P22', '#a6d854')
]

genes = list(set(L23_P22_marks[1].index).intersection(set(ics)))  # Add your genes here

plt_gene_idx(datasets, genes, title_colors=len(genes)*[sn.color_palette('Set2').as_hex()[1]],ncols=1, nrows=2)

In [ ]:
datasets = [
    (L23_FF, 'P22 10d-WD', '#e78ac3'),
    (L23_P22, 'P22', '#a6d854')
]

genes = list(set(L23_P22_marks[2].index).intersection(set(ics)))  # Add your genes here

plt_gene_idx(datasets, genes, title_colors=len(genes)*[sn.color_palette('Set2').as_hex()[2]],nrows=3, ncols=5)